# アルゴリズム

アルゴリズムの解法と解説.

Scalaの文法は適宜説明する.



整数

約数の全列挙

$N$の約数を全列挙するには、$N$を$k=1,2,\dots,\sqrt{N}$で割り、割れた場合は$k$と$N/k$を約数として記録すればいい.

これは36を例にすると以下の作業をやっているのと同じ.

1. <pre>1,               ,36</pre>
2. <pre>1,2,           18,36</pre>
3. <pre>1,2,3,      12,18,36</pre>
4. <pre>1,2,3,4,6,9,12,18,36</pre>

In [ ]:
def c(n:Int,d:Int=1,s:Set[Int]=Set()) :Set[Int] = {
    if(d>math.sqrt(n)){
        s
    }else {
        if(n%d==0){
            c(n,d+1,s++Set(d,n/d))
        }else {
            c(n,d+1,s)
        }
    }
}

RSA

$N=pq$,$E$を$p-1,q-1$の最小公倍数$L$との最大公約数が1となる素数、$D$を$DE mod L=1$となる整数とすると$\text{平文}^E mod N= \text{暗号文},\text{暗号文}^D mod N=\text{平文}$とすれば,十分大きな素数$p,q$を使うことで暗号文から元の素数$p,q$を復元するのは困難なので暗号化と復号化が理論上一方通行になる.

例えば、平文を$5$,$D=7,E=3,N=3\cdot11$とすると$5^3=125=33\cdot3+26,26^7=8031810176=243388187\cdot 33+5$である.

この場合、暗号文とD,Nがあれば暗号文から平文を得られる. また、Nを素因数分解することができれば平文,N,3から暗号文を生成することができる. 

今回は$N=33$なので簡単に$3,11$や$E,D$を得られるが、十分大きな素数$p,q$を使えば$N$から素因数$p,q$を得るのは現実的には困難になる.

In [ ]:
import scala.jdk.StreamConverters._
def encrypt(text:String,e:Int,n:Int)={
    text.codePoints.toScala(LazyList).map(i=>(BigInt(i).pow(e)%n).toInt).toList
}

def decrypt(encrypted:Seq[Int],d:Int,n:Int)={
    encrypted.map(i=>{(BigInt(i).pow(d)%n).toInt})
    
} 
val p = 307
val q = 109
val n = p*q
val l = (p-1)*(q-1)
val e = 13
val d= 15253
val en1 = encrypt("hello, 世界",e,n)
val en2 = encrypt("hello,🌍",e,n)
println(decrypt(en1,d,n).map(_.toChar).mkString)
println(decrypt(en2,d,n).map(_.toChar).mkString)

Scalaの文法について

Javaでは文字は16bitで表現する. 16bitに収まりきらない一部の文字はサロゲートペア(32bit)で一文字をあらわすので、`string.length`と実際の文字列の長さが一致しない場合がある. 

例えば🌍はサロゲートペア`\uD83C\uDF0D`であらわされる. `val s = 🌍;s.length //=>2`,`s.codepoints.toScala(LazyList).length//=>1`となる.

サロゲートペアを含む文字列はiteratorもずれる(iteratorはchar単位)ので`map`や`foreach`を使う際も注意が必要である.

```scala
"hello, 世界".foreach(println)
// h
// e
// l
// l
// o
// ,
// 
// 世
// 界

"🌍".foreach(println)
// ?
// ?
```

なお上の実装例では$n=p*q$までの整数しか扱えないので🌍$=127757_{(10)}=11111001100001101_{(2)}$を含む文字列は復号化に失敗し文字化けする.

`text.codePoints`は`java.util.Stream`型なので、そのままではScalaの`map`,`foldLeft`等が使えない. `import scala.jdk.StreamConverters._`することで`toScala`メソッドを使ってScalaのコレクションに変換できる.

文字列のコードポイントをべき乗するので`scala.math.pow`を`int`型に対して行うと桁数が足りず計算結果がおかしくなってしまうので、かわりに`BigInt.pow`を使う.

In [67]:
def gcd(n:Int,m:Int):Int={
    if(m==0) n
    else gcd(m,(n%m))
}

defined function gcd
res66_1: Int = 15

ugly number を 素因数に`2,3,5`以外を含む数とする.

例:
- isUgly(60 = 2 x 2 x 3 x 5) => false
- isUgly(14 = 2 x 7) => true
- isUgly(1) => false



In [74]:
def isUgly(n:Int)={
    if(n==0){
        n
    }else{
        val factors = Seq(2,3,5)
        var _n = n
        factors.foreach{f =>
          while (_n%f==0){
              _n = _n/f
          }
        }
        _n==1
    }
}

defined function isUgly
res73_1: AnyVal = true

線形代数

行列式

In [3]:
def det3x3(a:Array[Array[Double]]):Double= {
    var tmp:Double = 0.0
    for(i <- 0 to 2){
        tmp = tmp+ a(i%3)(0)*a((i+1)%3)(1)*a((i+2)%3)(2)
    }
    for(i <- 0 to 2){
        tmp = tmp - a((2-i)%3)(0)*a((4-i)%3)(1)*a((3-i)%3)(2)
    }
    tmp
}

defined function det3x3
res2_1: Double = 62.0

In [15]:
// a を 上三角行列に変形する. 破壊的変更
// note: a(0)(0)==0 でエラー. pivotが必要
// note: 非破壊的変更で出来ないか？
def upperTriangleMatrix(a:Array[Array[Double]]):Array[Array[Double]] = {
    val n = a.length
    
    for(i <- 0 to n-1){
        for(j <- 0 to n-1){
            if(i<j){
                val v = a(j)(i)/a(i)(i)
                for(k <- 0 to n-1){
                    a(j)(k) = a(j)(k)-a(i)(k)*v
                }
            }
        }
    }
    a  
}
// 対角成分の積
def diagonal(a:Array[Array[Double]]):Double = {
    var tmp = 1.0
    for(i <- 0 to  a.length-1){
        tmp=tmp*a(i)(i)
    }
    tmp
}
def detNxN(a:Array[Array[Double]]):Double = {
   diagonal( upperTriangleMatrix(a))
}
detNxN(Array(Array(2,-2,4,2),Array(2,-1,6,3),Array(3,-2,12,12),Array(-1,3,-4,4)))

defined function upperTriangleMatrix
defined function diagonal
defined function detNxN
res14_3: Double = 120.0

ハンガリー法

In [1]:

val n = 4
val testArray: Array[Array[Double]] = Array(
  Array(5, 4, 7, 6),
  Array(6, 7, 3, 2),
  Array(8, 11, 2, 5),
  Array(9, 8, 6, 7)
)
val rowMins = testArray.map(a => a.min)

val subtractRowMins =
  testArray.zip(rowMins).map { case (row, rowMin) => row.map(_ - rowMin) }
val colMins = subtractRowMins.foldLeft(Array.fill[Double](testArray.length)(Double.MaxValue)){case (mins,row)=>
  row.zip(mins).map{case (value,maybeMin)=> if(value < maybeMin) value else maybeMin}
}
val a = subtractRowMins.transpose.zip(colMins).map{case (col,colMin)=>col.map(_-colMin)}.transpose
def where(mat:Seq[Seq[Double]]):Seq[(Int,Int)] = {

  mat.zipWithIndex.flatMap{ case (row,rowIdx)=>
    row.zipWithIndex.foldLeft(Seq():Seq[(Int,Int)]){case (acc,(v,colIdx))=>
      if(v==0.0){
        acc.appended((rowIdx,colIdx))
      }else{
        acc
      }
    }
  }
}

n: Int = 4
testArray: Array[Array[Double]] = Array(
  Array(5.0, 4.0, 7.0, 6.0),
  Array(6.0, 7.0, 3.0, 2.0),
  Array(8.0, 11.0, 2.0, 5.0),
  Array(9.0, 8.0, 6.0, 7.0)
)
rowMins: Array[Double] = Array(4.0, 2.0, 2.0, 6.0)
subtractRowMins: Array[Array[Double]] = Array(
  Array(1.0, 0.0, 3.0, 2.0),
  Array(4.0, 5.0, 1.0, 0.0),
  Array(6.0, 9.0, 0.0, 3.0),
  Array(3.0, 2.0, 0.0, 1.0)
)
colMins: Array[Double] = Array(1.0, 0.0, 0.0, 0.0)
a: Array[Array[Double]] = Array(
  Array(0.0, 0.0, 3.0, 2.0),
  Array(3.0, 5.0, 1.0, 0.0),
  Array(5.0, 9.0, 0.0, 3.0),
  Array(2.0, 2.0, 0.0, 1.0)
)
defined function where

In [3]:
val zeros = where(a.map(_.toSeq).toSeq)

zeros: Seq[(Int, Int)] = ArraySeq((0, 0), (0, 1), (1, 3), (2, 2), (3, 2))

In [12]:
// 1. res1から 0 を行として選ぶ
// 選んだ0 以外から0を選ぶ
val positions = zeros.foldLeft((Seq(),Seq()):(Seq[Int],Seq[Int])){case ((row,col),(x,y))=>
  if(!row.contains(x) && !col.contains(y)){
      (row.appended(x),col.appended(y))
  }else{
      (row,col)
  }
}
// 最小positions._1.length || positions._2.length で 0 をすべて消せる
(zeros.groupBy(_._1).toSeq,zeros.groupBy(_._2).toSeq) 

positions: (Seq[Int], Seq[Int]) = (List(0, 1, 2), List(0, 3, 2))
res11_1: (Seq[(Int, Seq[(Int, Int)])], Seq[(Int, Seq[(Int, Int)])]) = (
  List(
    (0, ArraySeq((0, 0), (0, 1))),
    (1, ArraySeq((1, 3))),
    (2, ArraySeq((2, 2))),
    (3, ArraySeq((3, 2)))
  ),
  List(
    (0, ArraySeq((0, 0))),
    (1, ArraySeq((0, 1))),
    (2, ArraySeq((2, 2), (3, 2))),
    (3, ArraySeq((1, 3)))
  )
)

In [10]:
// もっともゼロがおおい行または列から順に選んで消していく


: 

## ソート

選択ソート
1. i=0 とする
2. i 番目から配列の最後尾まで全ての要素を確認し、最小の要素と i 番目の要素を入れ替える
3. i を 1 増やして、i が配列の長さ以上だったら終了。そうでなければ 2. に戻る

In [14]:
// search minimum value from focus to the last element of array 
def min(arr:Array[Int],focus:Int=0,maybeMin:(Int,Int)=(-1,Int.MaxValue)): (Int,Int) = {
    if(focus==arr.length){
        maybeMin
    }else {
      if(arr(focus)< maybeMin._2) min(arr,focus+1,(focus,arr(focus)))
      else min(arr,focus+1,maybeMin)
    }
}
// select sort: repetitively exec  `min` while incrementing focus

def selectSort(arr:Array[Int],focus:Int=0):Array[Int] = {
    if(focus == arr.length){
        arr
    }else {
       val (f,m) = min(arr,focus)
        arr(f) =  arr(focus)
        arr(focus) = m
        selectSort(arr,focus+1)
    }
}

selectSort(Array(5,4,3,2,6))

defined function min
defined function selectSort
res13_2: Array[Int] = Array(2, 3, 4, 5, 6)

#### バブルソート


アイディアは次のようなものである.

1. i番目の要素とi+1番目の要素を比較し、大きい方を右に動かす処理をリスト全体に対して行う. リスト全体を通過した時点で一番右の要素はリストの中で最大の要素になる.
1. リストをソート済みの領域(一回目が終わった時点で一番右のひとつ)とソート前の領域に分けてソート前の領域に対して同様の処理を繰り返す.

計算量は$O(N^2)$

以下に`for`ループを使った実装`bubbleSort`と再帰を使った実装`bubbleSortRecursively`を書く.
`bubbleSort`では内側のループの範囲を`arr.length-2`から`i`をひいて狭めることで、`bubbleSortRecursively`では配列のソート前の領域を再帰的に渡すことで実装している.

In [ ]:
def bubbleSort[T](arr:Array[T])(implicit comparator:Ordering[T]):Seq[T]={
    for(i <- 0 to arr.length-1){
        for(j <- 0 to arr.length-2 -i){
            if(comparator.compare(arr(j),arr(j+1))>0){
                val tmp=arr(j)
                arr(j)=arr(j+1)
                arr(j+1)=tmp

            }
        }
    }
    return arr
}

Scalaの文法解説

型パラメタ、暗黙の引数

1行目の`bubbleSort[T]`の`[T]`は型パラメタと呼ばれる概念である.JavaやKotlinではジェネリクスとも呼ばれる機能で`T`に応じて`arr: Array[T]`の型が決まる. 

※なお、帰り値の型が`Seq[T]`になっているのは事情があるが、それは次のセクションで説明する. 

例えば、`bubbleSort(Array(5,4,2,4,1))`を呼び出した場合、型推論から`T=Int`になり、`arr`は`arr[Int]`として扱われる.

型パラメタを使うことで`Int`,`Float`,`Double`など、さまざまな型についてそれぞれの実装を抽象化できる.

`implicit comparator: Ordering[T]`は`implicit`が暗黙の引数, `Ordring`が型クラス`TypeClass`と呼ばれる機能である.  型クラス`Ordering[T]`を与えることで4行目の比較処理を様々な型について対応させることができる.

Scalaのプリミティブ型については比較用の型クラスが既に定義されているのでいちいち`comparator`を渡さなくても必要な型クラスを`implicit`スコープから探してくれる.

例えば、自作クラス`Student(no:Int,name:String)`があったとしよう. このとき`Student`の配列`Array[Student]`をソートすることを考える.

よくあるオブジェクト指向のコードでは(`class Student extends Comparable`といった風に)`Student`で比較処理の機能を担うクラスを継承し比較演算子を`override`する方法が用いられる.

これは比較すべきエンティティに比較機能を持たせる方針である.

型クラスを利用する場合、比較すべきエンティティではなく比較処理に比較ルール(`Ordering[Student]`)を渡す方針をとる.

```scala
val studentOrdering = new Ordering[Student]{
    override def compara(one:Student,another:Student):Int = {
      // ここに比較処理を書く
    }
}

bubbleSort(Array(Student(5,"foo"),Student(1,"bar"),Student(3,"buzz")))(studentOrdering)
```

注目すべきは、このとき`bubbleSort`も`Student`も一切コードに変更が加えられていないということである. このように型クラスを使う場合,責務をうまく分離してコードを書くことができる.

さて、`bubbleSort`の第二引数`ordering`は`T`が決まれば決まるのだから`Array[Student]`をソートするたびに`studentOrdering`を渡すのは冗長である. ここで活躍するのが先に述べた`implicit`である.

一行目を以下のように書き換えることで`implicit`の探索スコープに`studentOrdering`が入るのでいちいち`studentOrdering`を渡してやる必要がなくなる.

```scala
implicit val studentOrdering = new Ordering[Student]{
// ... 中略
bubbleSort(Array(Student(5,"foo"),Student(1,"bar"),Student(3,"buzz")))
```

続いて、再帰を使った実装をする.

再帰を使ってコードを書くと`n=k`と`n=k+1`(`n=k`と`n=k-1`と考えても良い)の関係および終端条件を考えるだけで処理を記述できる.

バブルソートを、$A(n) : \text{整列前の長さnの配列}\rightarrow A^*(n) : \text{整列後の長さnの配列}$という関数$f$とみなすと、
$A$の最大値を$M$として$f(A(n))=f(A(n-1))+M,f(A(1))=A(1)$と書ける.(ただし`+`は配列の末尾への要素の追加をあらわす)

またすべての繰り返し処理は再帰で表現可能なので以下の実装では繰り返し処理に`for`文をつかわないで実装する. 

In [1]:
import scala.reflect.ClassTag
def bubbleSortRecursively[T: ClassTag : Ordering](arr:Array[T]):Array[T]= {
     if(arr.length==1){
         arr
     }else{
         incrementally(0,arr.length-1){i=>
           if(implicitly[Ordering[T]].compare(arr(i),arr(i+1))>0){
                 val tmp = arr(i)
                 arr(i) = arr(i+1)
                 arr(i+1)=tmp
             }
         }
         bubbleSortRecursively(arr.slice(0,arr.length-1)) :+ arr.last
     }
}

def incrementally[T](from:Int,until:Int)(f:Int=>T):Unit={
    require(from<=until)
    if(from==until){
        return
    }else{
        f(from)
        incrementally(from+1,until)(f)
    }
}
bubbleSortRecursively(Array(6,5,1,2,7,3,1,7,9,1,6,3,5,1,61))

: 


#### マージソート

> マージソートは、ソートのアルゴリズムで、既に整列してある複数個の列を1個の列にマージする際に、小さいものから先に新しい列に並べれば、新しい列も整列されている、というボトムアップの分割統治法による。 
> [wikipedia: マージソート](https://www.google.com/search?q=%E3%83%9E%E3%83%BC%E3%82%B8%E3%82%BD%E3%83%BC%E3%83%88&oq=%E3%83%9E%E3%83%BC%E3%82%B8%E3%82%BD%E3%83%BC%E3%83%88&aqs=chrome..69i57j0l6j69i61.3319j0j7&sourceid=chrome&ie=UTF-8)



実装は配列を分割フェーズと併合フェーズに分かれる. 分割フェーズでリストを長さ2になるまで分割し順序を整理したあとそれぞれのリストを結合する.

結合する際は二つの配列の先頭同士を比較し、小さいほうから順に新しい配列につめる. 例えば分割された配列が`Array(1,3,9),Array(2,4,8)`なら`1`,`2`,`3`,`4`...の順で新しい配列に格納する.

In [60]:
import scala.collection.mutable.ArrayBuffer
def mergeSort[T](seq:Seq[T])(implicit comparator:Ordering[T]):Seq[T]={
    if(seq.length==2){
        if(comparator.compare(seq(0),seq(1))>0){
            seq.reverse
        }else{
            seq
        }
    }else if(seq.length==1){
        seq
    }else{
        val (left,right) = split(seq)
       val sorted1= mergeSort(left)
        val sorted2=mergeSort(right)
        merge(sorted1,sorted2).toSeq
    }
    
}

def split[T](seq:Seq[T]):(Seq[T],Seq[T])={
    (seq.take(seq.length/2),seq.drop(seq.length/2))
}

def merge[T](seq1:Seq[T],seq2:Seq[T],tmp:ArrayBuffer[T]=ArrayBuffer[T]())(implicit comparator: Ordering[T]):ArrayBuffer[T]={
    if(seq1.isEmpty && seq2.isEmpty){
        return tmp

    }else if(seq1.isEmpty & !seq2.isEmpty){
        tmp.append(seq2(0))
        merge(seq1,seq2.tail,tmp)
    }else if(seq2.isEmpty & !seq1.isEmpty){
        tmp.append(seq1(0))
        merge(seq1.tail,seq2,tmp)
    }else{
    if(comparator.compare(seq1(0),seq2(0))>0){
          tmp.append(seq2(0))
          merge(seq1,seq2.tail,tmp)
      }else{
          tmp.append(seq1(0))
          merge(seq1.tail,seq2,tmp)
      } 
    }
}

import scala.collection.mutable.ArrayBuffer

defined function mergeSort
defined function split
defined function merge

### ヒープソート
wip
### クイックソート
wip

greedy

おつりの枚数の最小化

複数の支払う金額が与えられる。それぞれに対して1000円を支払った時、お釣りに含まれる硬貨の枚数の最小値を求めよ。
お釣りに使える硬貨は、500, 100, 50, 10, 5, 1円がそれぞれ十分な数ある。

In [22]:
// おつりpが与えられたとき, p を越えない最大の硬貨cで支払う,  p-c に対して同様の処理を繰り返す
// 持っている効果の種類を降順にしたリストを用意する
def calcChange(p:Int,changes:Int=0,coins:List[Int]=List(500, 100, 50, 10, 5, 1)):Int = {
   coins match {
       case head :: tail if head > p  => calcChange(p,changes,tail)
       case cs @ head :: tail if head < p  => calcChange(p-head,changes+1,cs)
       case head :: tail if head == p => changes+1
   }
}


defined function calcChange
res21_1: Int = 5

## 動的計画法

1. currencies=`{1, 2, 4, 8, 16, 32}`,が与えられ、`currencies[i]`を$a_i\in \mathbb{N}(0 \in \mathbb{N})$枚つかって$N$円を支払うときに支払う通貨の枚数の総和$S(N)=\{ \sum_{j=0}^{n-1}a_j \}$を最小にするような$S(N)=S^*(N)$を求めよ.

参考: N=63のとき$S^*(N)=6$

In [1]:
import scala.collection.mutable.ArraySeq
val N:Int =63
val currencies:Seq[Int] =Seq(1,2,4,8,16,32)
/*S(i)*/
val SHistory: ArraySeq[Int]= ArraySeq.fill(N+1)(Int.MaxValue)
SHistory(0)=0
currencies.foreach{c => SHistory(c)=1}

import scala.collection.mutable.ArraySeq

N: Int = 63
currencies: Seq[Int] = List(1, 2, 4, 8, 16, 32)
SHistory: ArraySeq[Int] = ArraySeq(
  0,
  1,
  1,
  2147483647,
  1,
  2147483647,
  2147483647,
  2147483647,
  1,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  1,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  1,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
  2147483647,
...

$N$円払うときの通貨の枚数の総和$S(N)$は$currency_k\in currencies$として,$min(S(N-currency_k)+1,S(N))$と書ける.

たとえば、$N=15$なら、
- $S(15)=S(11)+1,S(11)=S(10)+1,S(10)=S(8)+1,S(8)=1$
- $S(15)=S(13)+1,S(13)=S(11)+1,S(11)=S(10)+1,\dots$
- $S(15)=S(14)+1,S(14)=S(12)+2,S(12)=S(10)+1,\dots$

とさまざまな$S(15)$があり得るが、$S(N)$の初期値を十分大きな値に設定し$min(S(N-currency_k)+1,S(N))$で比較することで$S^*(N)$が得られる.(S(N)の値は過去に$S(N)$を訪れたことがあればその時の値が、最適な方法が見つからない場合は十分大きな値のままになる.)

この問題のように$S(11)\rightarrow S(15),S(13)\rightarrow S(15),S(14)\rightarrow S(15),\dots$など、$S(N)$にいたる経路が複数ある場合、つまり異なる経路$\alpha,\beta,\gamma,\dots$をたどって目的地にたどり着いたときの結果(総コスト)同士を比較したいときにこの書き方ができる. (この書き方は$S(N)$へ至る経路を考えるので**貰うDP**と呼ばれることもある. 各$N$について$S(N)$から遷移可能な経路(この例の場合`S(n+currency)`)を更新する**配るDP**と呼ばれる書き方もできる.)

In [2]:
import scala.{math => _Math}
def S(N:Int):Int = {  
      for(n <- 1 to N){
           currencies.foreach{ currency =>
             if(n >= currency){
                  SHistory(n) = _Math.min( SHistory(n-currency)+1,SHistory(n))
             }
           }
      }
  SHistory(N)
}

S(63)

import scala.{math => _Math}

defined function S
res1_2: Int = 6

類題

$h \in \mathbb{N}$をパラメタにもつN個のノードがある.ノード$i$は`nodes:List[Int]`の$i$番目の要素をパラメタ$h_i$として与えらえる. ノード$n_i$からノード$n_{i+1}$への移動コストは$|h_i- h_{i+1}|$、ノード$n_i$からノード$n_{i+2}$への移動コストは$|h_i - h_{i+2}|$である.

ノード1からスタートするとき、ノード1からノードNへ移動する最小コストを求めよ.

配るDPで実装する.

In [34]:
import scala.collection.mutable.ArrayBuffer
def shortestPathDP(nodes:Seq[Int]) = {
    var history = ArrayBuffer.fill(nodes.length+2)(Int.MaxValue)
    history(0)=0
    history.toArray
    
    for(i <- 0 to nodes.length -1){
        if(i==nodes.length-2){
            history(i+1)= math.min(history(i+1),history(i)+(nodes(i)-nodes(i+1)).abs)

        }else if(i<nodes.length -2){
             history(i+1)= math.min(history(i+1),history(i)+(nodes(i)-nodes(i+1)).abs)
            history(i+2)= math.min(history(i+2),history(i)+(nodes(i)-nodes(i+2)).abs)
        }     
    }
    history(nodes.length-1)
}

import scala.collection.mutable.ArrayBuffer

defined function shortestPathDP

In [35]:
shortestPathDP(Seq(2,9,4,5,1,6,10))

res34: Int = 8

類題

$n=k+1$で$n=k$のときの情報が必要な場合.

$i$日目の予定をA: 効用$a_i$,B: 効用$b_i$,C: 効用$c_i$から選ぶとき$N$日間の効用の総和$S$を最大にするような予定を求める.

ただし、$k$日目に予定$\alpha$を選んだとき$k+1$日目には同じ予定を選べない.

方針

$N$日間までの効用の総和が$S_N'$かつ$N$日目に予定$\alpha$を選んだことを二次元配列を使って表現する.

つまり`history[N][0]`,`history[N][1]`,`history[N][2]`にそれぞれ$N$日目に予定$0=A,1=B,2=C$を選んだときの最大効用$S'$を記録する.

In [15]:
def solution(utilities:Seq[Array[Int]])={
    // history: i日目までの和. ((0,0,0),(a_i,b_i,c_i),...)
    var history :Array[Array[Int]]= Array.fill(100010)(Array.fill(3)(Int.MinValue))
        // e.g. utilities = Seq((1,5,2),(3,5,4),(.,.,.),...)
    history(0) =  Array.fill(3)(0)
    
    for(i <- 0 to utilities.length -1){
            history(i).zipWithIndex.foreach{ case ( sum_of_utilities,action_of_the_day_before) =>
                (Set(0,1,2) - action_of_the_day_before).foreach {action =>
                   history(i+1)(action) = math.max(sum_of_utilities + utilities(i)(action),history(i+1)(action))
                }

            }
        
    }
    
}

defined function solution

## 探索


### 全探索 + 枝刈
Longest Substring Without Repeating Characters
> 与えられた文字列に対して、それぞれの文字を繰り返さない最長の部分文字列の長さを求めなさい。
> abcabcbb -> abc
> bbbbbbb -> b
> pwwkew wke -> 3

$0 \leq j < k \leq length-1$を満たすすべての$j,k$について`Str.substring(j,k)`がユニークかどうか確かめる.

In [2]:
var longest = Set[Char]()
def lswrc(str:String) = {
    for(j <- 0 to str.length-1){
        for (k <- j+1 to str.length -1){
            val s = str.substring(j,k)
            val set = s.toSet
            if(set.size == s.length){
                if(longest.size < s.length) {
                    longest = s.toSet
                }
            }            
        }
    }
}
lswrc("pwwkew")

longest: Set[Char] = Set('w', 'k', 'e')
defined function lswrc

尺取り法
- 区間 [left, right) が「条件」を満たすなら、それに含まれる区間も「条件」を満たす
- 区間 [left, right) が「条件」を満たすなら、それを含む区間も「条件」を満たす


In [21]:
var pair = (0,Set[Char]())
def lswrc2(str:String,left:Int=0,right:Int=0,longest:Set[Char]=Set()) :Set[Char]= {
    if(right == str.length-1){
        longest
    }else if(longest.contains(str(right))){
            lswrc2(str,left+1,right,longest - str(left))
    }else {
        if(pair._1 < longest.size + 1){
            pair = (longest.size+1,longest + str(right))
        }
        lswrc2(str,left,right+1,longest+str(right))
    }
}
lswrc2("pwabcwkewst")
pair

pair: (Int, Set[Char]) = (6, HashSet('e', 'a', 'b', 'c', 'w', 'k'))
defined function lswrc2
res20_2: Set[Char] = HashSet('e', 's', 'w', 'k')
res20_3: (Int, Set[Char]) = (6, HashSet('e', 'a', 'b', 'c', 'w', 'k'))

効率的なsliding window

In [ ]:
def esw(s:Seq[Char],found:Map[Char,Int]=Map(),longest:(Int,Seq[Char])=(0,Seq()),left:Int=0,right:Int=0):(Int,Seq[Char])={
    println(left,right)
    if(right>=s.length){
       return  longest
    }else if(left >=s.length){
         return longest
    }else if(found.contains(s(right))){
        esw(s,found.updated(s(right),right),longest,math.max(left,found(s(right))+1),math.max(right+1,found(s(right))+1))
    }else {
        esw(s,found.updated(s(right),right),{if(right-left+1>longest._1) (right-left+1,s.slice(left,right+1)) else longest},left,right+1)
    }
}
//esw("pwabcwkewst")


類題

> 長さ n の正の整数列$a_1,a_2,…,a_n$と整数$x$が与えられる。整数列の連続する部分列で、その総和が x 以下となるものを数え上げよ (実際の出題は Q 個のクエリがあって各クエリごとに xが与えられる)。
- $1≤n≤10^5$
- $1≤Q≤500$
- $1≤a_i≤10^9$
- $1≤x_j≤10^{14}$

例 n= 6 、x=12, a=(5,3,8,6,1,4)ならば 11

In [ ]:
// メモ 間違い. 区間の数を数えるだけならsetで記録する必要はない
// 尺取り法でleft,rightをうごかしつつ sum += [left,right+1).len すれば区間の数を数え上げられる.
def s(a:Seq[Int],x:Int,set:Set[Seq[Int]]=Set(),left:Int=0,right:Int=0):Set[Seq[Int]]= {
    if(right >= a.length && left >= a.length){
        set
    }else if(a.slice(left,right+1).sum <= x){
        if(right==a.length){
            val _set = (left to a.length-1).foldLeft(set){case (acc,i)=>
              acc+a.slice(left,i+1)
            }.toSet
            s(a,x,_set,if(right==a.length) left+1 else left,math.min(a.length,right+1))

        }else {
            val _set = extractSubset(a.slice(left,right+1))
            
            s(a,x,set++_set,if(right==a.length) left+1 else left,math.min(a.length,right+1))
        }
    }else {
        s(a,x,set+a.slice(left+1,right),left+1,math.min(a.length,math.max(right,left+1)))
    }
}
s(Seq(4,6,7,8,1,2,110,2,4,12,3,9),25).filterNot(_.isEmpty)
def extractSubset(seq:Seq[Int]):Set[Seq[Int]]= {
        (0 to seq.length-1).flatMap{i =>
          (i to seq.length-1).foldLeft(Set[Seq[Int]]()){case (acc,j)=>
          
            acc+seq.slice(i,j+1)
          }
        }.toSet       
}


長さ$N$ の数列$a_1,a_2,...,a_N$と整数$S$が与えられる.
要素の総和が$S$以上となる連続する部分列のうち、最も短いものの長さ（smallest window length）を求めてください。ただし、そのような部分列が存在しない場合は 0 と報告してください。

In [146]:
val a = Seq(1,2,1,2,3,2)
val s = 4
// expect 2
def sol(a:Seq[Int],s:Int,sum:Int=0,min:Int=Int.MaxValue,left:Int=0,right:Int=0):Int = {
    if(right==a.length-1){
        min
    }else if(sum+a(right)>=s){
        sol(a,s,sum-a(left),math.min(min,right-left+1),math.min(a.length-1,left+1),right)
    }else{
        sol(a,s,sum+a(right),min,left,math.min(a.length-1,right+1))
    }
}

a: Seq[Int] = List(1, 2, 1, 2, 3, 2)
s: Int = 4
defined function sol
res145_3: Int = 1

bit全探索による順列の計算

todo

ハノイの塔は漸化式をたてれば解けるが、仮に漸化式を思いつかなくてもちょうど我々がパズルを解くうまいやり方が見つからないで手当たり次第に試行錯誤するように、探索を使えば解を求めることができる場合もある.
### 幅優先探索

初期状態$\begin{vmatrix}2 & 3 & 4 \\ 7 & 1 & \emptyset \\ 6 & 8 & 5 \end{vmatrix}$からうまく$\emptyset$をスライドさせて$\begin{vmatrix}1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & \emptyset \end{vmatrix}$にするパズル(8パズルと呼ばれる)を幅優先探索を使って解いてみる.  幅優先探索はQueueを使って実装できる. これは木構造を横に倒してみると理解しやすい.


直前の状態から次にとりうる操作が制限できるのでうまく枝刈りをしていこう.  例えば、$\emptyset$を右に動かした直後に左に動かすと元の状態に戻ってしまうのでその操作は選択肢から除外したほうがいい.

In [12]:
import scala.collection.mutable.Queue
val EMPTY = Int.MinValue
val toBe:ArraySeq[ArraySeq[Int]]=ArraySeq(ArraySeq(1,2,3),ArraySeq(4,5,6),ArraySeq(7,8,EMPTY))

import scala.collection.mutable.Queue

EMPTY: Int = -2147483648
toBe: ArraySeq[ArraySeq[Int]] = ArraySeq(
  ArraySeq(1, 2, 3),
  ArraySeq(4, 5, 6),
  ArraySeq(7, 8, -2147483648)
)

In [ ]:
CNT=0
def solution(history:Queue[((Int,Int),ArraySeq[ArraySeq[Int]],Int)],toBe:ArraySeq[ArraySeq[Int]]):ArraySeq[ArraySeq[Int]]={
    CNT=CNT+1
    val h = history.dequeue
    h match {
        case (emptyAt,state,_) if state == toBe =>state
        case (emptyAt,state,prevAction) =>
          history.appendAll(nextCandidates(emptyAt,state,prevAction))
          solution(history,toBe)
    }
}
val upward = (emptyAt:(Int,Int),state:ArraySeq[ArraySeq[Int]])=>Queue(((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0)       )
val rightward = (emptyAt:(Int,Int),state: ArraySeq[ArraySeq[Int]])=>Queue(((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1))
val downward =  (emptyAt:(Int,Int),state:ArraySeq[ArraySeq[Int]])=>Queue(((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),2)        )
val leftward  = (emptyAt:(Int,Int),state:ArraySeq[ArraySeq[Int]])=> Queue(((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3))

def nextCandidates(emptyAt:(Int,Int),state: ArraySeq[ArraySeq[Int]],prevAction:Int):Queue[((Int,Int),ArraySeq[ArraySeq[Int]],Int)]={
    val rowEndAt = state.length -1
    val colEndAt = state(0).length -1
    // prevAction: ↑:0,→:1,↓:2,←:3
    if(emptyAt._1==0){
        if (emptyAt._2==0){
            if(prevAction==0){
                return rightward(emptyAt,state)
            }else {
                 return downward(emptyAt,state)
            }
        }else if (emptyAt._2==colEndAt){
            if(prevAction==1){
                return downward(emptyAt,state)
            }else {
             return leftward(emptyAt,state)
            }
        }else {
            if(prevAction==1){
                 return Queue(
                ((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),2),      
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1)
            )
            }else if(prevAction==3){
                 return Queue(
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
                ((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),2),      
            )
            }else{
                return Queue(
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1)
            )
            }
        }
    }else if(emptyAt._1==rowEndAt){ 
         if (emptyAt._2==0){
             if(prevAction==2){
                  return Queue(
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1),
            )
             }else{
                 return upward(emptyAt,state)
             } 
 
        }else if (emptyAt._2==colEndAt){
             if(prevAction==1){
                 return Queue(   
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),1)
            )
             }else {
                 return Queue(   
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
            )
             }
        }else {
             if(prevAction==1){
                 return Queue(
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0),      
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1)
            )
             }else if(prevAction==3){
                 return Queue(
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0),      
            )
             }else {
                 return Queue(
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1)
            )
             }
        }
    }else{
        if (emptyAt._2==0){
            if(prevAction==3){
                return Queue(
                ((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),2),       
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0)      
            )
            }else if(prevAction==0){
                return Queue(
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1),
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0)      
            )
            }else{
                return Queue(
                ((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),2),       
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1),
            )
            }
        }else if (emptyAt._2==colEndAt){
            if(prevAction==1){
                return Queue(   
                ((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),2),       
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0)   
            )
            }else if(prevAction==0){
                return Queue(   
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0)   
            )
            }else {
                return Queue(   
                ((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),0),       
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
            )
            }
        }else {
            if(prevAction==0){
                return Queue(
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1),
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0)    
            )
            }else if(prevAction==1){
                return Queue(
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1),
                ((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),2),    
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0)    
            )
                
            }else if (prevAction==2){
                 return Queue(
                ((emptyAt._1,emptyAt._2+1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2+1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2+1))),1),
                ((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),2),    
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
            )
            }else{
                return Queue(
                ((emptyAt._1+1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1+1)(emptyAt._2))).updated(emptyAt._1+1,state(emptyAt._1+1).updated(emptyAt._2,EMPTY)),2),    
                ((emptyAt._1,emptyAt._2-1),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2-1,EMPTY).updated(emptyAt._2,state(emptyAt._1)(emptyAt._2-1))),3),
                ((emptyAt._1-1,emptyAt._2),state.updated(emptyAt._1,state(emptyAt._1).updated(emptyAt._2,state(emptyAt._1-1)(emptyAt._2))).updated(emptyAt._1-1,state(emptyAt._1-1).updated(emptyAt._2,EMPTY)),0)    
            )
            }
        }
        
    }
}

In [ ]:
val asIs:ArraySeq[ArraySeq[Int]]=ArraySeq(ArraySeq(2,3,4),ArraySeq(7,1,EMPTY),ArraySeq(6,8,5))
val emptyAt :(Int,Int)=(1,2)
val history:Queue[((Int,Int),ArraySeq[ArraySeq[Int]],Int)] = Queue((emptyAt,asIs,-1))
solution(history,toBe)
print(CNT)

In [1]:
import $exec.helpers.scala.model.GraphHelpers,GraphHelpers.GraphNode

val graph = Seq(
  GraphNode(1,2,3),
    GraphNode(2,3,4),
    GraphNode(3,5),
    GraphNode(4,6),
    GraphNode(5,6),
    GraphNode(6)

)

import $exec.$                               ,GraphHelpers.GraphNode


graph: Seq[GraphNode] = List(
  GraphNode(id = 1, nexts = ArraySeq(2, 3)),
  GraphNode(id = 2, nexts = ArraySeq(3, 4)),
  GraphNode(id = 3, nexts = ArraySeq(5)),
  GraphNode(id = 4, nexts = ArraySeq(6)),
  GraphNode(id = 5, nexts = ArraySeq(6)),
  GraphNode(id = 6, nexts = List())
)

In [4]:
val graph2 = Seq(
GraphNode(1,2),
    GraphNode(2,4),
    GraphNode(3),
    GraphNode(4,3)
)

graph2: Seq[GraphNode] = List(
  GraphNode(id = 1, nexts = ArraySeq(2)),
  GraphNode(id = 2, nexts = ArraySeq(4)),
  GraphNode(id = 3, nexts = List()),
  GraphNode(id = 4, nexts = ArraySeq(3))
)

In [5]:
import scala.collection.mutable.Stack
//  k 番目のノードの情報は Array の k-1 番目に格納される

var arrives :Array[Int] = Array.fill(graph2.length)(-1)
var returns :Array[Int] = Array.fill(graph2.length)(-1)
var s:Stack[Int] = Stack()
// 行きがけ(pushしたときに)に1,2,3,...
// 帰りがけ(popしたときに)に...11,12
var cnt=1
s.push(1)
arrives(0) = cnt
while(!s.isEmpty){
    println(s)
    val top = s.top
    // 次に行けるならpop(反転)しないでpushする
    graph2(top-1).nexts.foreach {n=>
        if(arrives(n-1)== -1){
            cnt+=1
            arrives(n-1) = cnt
            s.push(n)
        } 
    }  
    // ここでstackの中身が変わっている. 先頭は最後に訪問したところ
    
    // push(訪問)したあとそこから次に行けないならpop(反転する)する
    if(!graph2(s.top-1).nexts.exists(n=> arrives(n-1) == -1)){
        cnt+=1
        returns(s.top-1) = cnt
        s.pop
    }
}
println(arrives,returns)




Stack(1)
Stack(2, 1)
Stack(4, 2, 1)
Stack(4, 2, 1)
Stack(2, 1)
Stack(1)
([I@2113b56,[I@9e6e8f7)


import scala.collection.mutable.Stack
//  k 番目のノードの情報は Array の k-1 番目に格納される


arrives: Array[Int] = Array(1, 2, 4, 3)
returns: Array[Int] = Array(8, 7, 5, 6)
s: Stack[Int] = Stack()
cnt: Int = 8
res4_5: Stack[Int] = Stack()

In [15]:
val n=graph.length
var arrives = Array.fill(n)(-1)
var returns = Array.fill(n)(-1)
val tree = graph
var count = 0
def dfsByRec(currentNode:Int=1):Unit={
    // ここが行きがけ
    count+=1
    arrives(currentNode-1) = count
    if(tree(currentNode-1).nexts.exists(n=> arrives(n-1) == -1)){
      tree(currentNode-1).nexts.foreach{n=>
          // filter でやると、他の経路から来た場合で変になる
        if(arrives(n-1)== -1){
            dfsByRec(n)
        }
      }
    }
    // 探索しなかった/し終わったならこっち
    count+=1
    returns(currentNode-1)=count
    
    
}
dfsByRec(1)


n: Int = 6
arrives: Array[Int] = Array(1, 2, 3, 9, 4, 5)
returns: Array[Int] = Array(12, 11, 8, 10, 7, 6)
tree: Seq[GraphNode] = List(
  GraphNode(id = 1, nexts = ArraySeq(2, 3)),
  GraphNode(id = 2, nexts = ArraySeq(3, 4)),
  GraphNode(id = 3, nexts = ArraySeq(5)),
  GraphNode(id = 4, nexts = ArraySeq(6)),
  GraphNode(id = 5, nexts = ArraySeq(6)),
  GraphNode(id = 6, nexts = List())
)
count: Int = 12
defined function dfsByRec

1,3,5からなるN桁の数字

深さNの深さ優先探索で求められる. 深さ優先探索は再帰を使ってもStackを使っても良い.

以下、再帰を使った実装とスタックを使った実装を書く.

In [9]:
import scala.collection.mutable.ArrayBuffer
val candidates = Array("1","3","5")
def solution(n:Int,result:Array[String]=candidates):Array[Int]={
    if(n==1){
        result.map(_.toString).map(_.toInt)
    }else{
        val next = result.foldLeft[ArrayBuffer[String]](ArrayBuffer()){case (acc,s)=>
            candidates.foreach{c=> acc.append(s+c)}
            acc
        }
        solution(n-1,next.toArray)
    }
}

import scala.collection.mutable.ArrayBuffer

candidates: Array[String] = Array("1", "3", "5")
defined function solution

In [10]:
import scala.collection.mutable.ArrayStack
val candidates = ArrayStack("1","3","5")
def solution2(n:Int)={
    var stack = ArrayStack[String]()
    var result = ArrayBuffer[String]()
    stack.pushAll(candidates)
    while(!stack.isEmpty){
        val top :String= stack.pop
        result.append(top)
        if(top.length<n){
            val nextStates=candidates.map(c=>c+top)
          stack.pushAll(nextStates)
        }  
    }
    result.toArray
}

import scala.collection.mutable.ArrayStack

candidates: collection.mutable.Stack[String] = Stack("1", "3", "5")
defined function solution2

有向グラフの探索

> 最初の行に G の頂点数 n が与えられます。続く n 行で各頂点 u の隣接リストが以下の形式で与えられます：
>
> $ukv_1 v_2 ... v_k$
>
> u は頂点の番号、k は u の出次数、$v_1v_2\dots v_k$　は u に隣接する頂点の番号を示します。

In [196]:
import scala.collection.mutable.{ArrayBuffer,Queue}
val input = """1 2 2 4
2 1 4
3 0
4 1 3"""
val nodes = Seq(GraphNode(1,Seq(2,4)),GraphNode(2,Seq(4)),GraphNode(3,Seq()),GraphNode(4,Seq(3)))
def solution()={
    var result:ArrayBuffer[Int] = ArrayBuffer.fill(4)(-1)
    var q = Queue[GraphNode]()
    result(0)=0
    q.append(nodes(0))
    while(!q.isEmpty){
        val top = q.dequeue
        top.nexts.foreach{n => 
            println(result)
          if( result(n-1) == -1 || result(top.number-1)+1<result(n-1)){
              result(n-1)=result(top.number-1)+1
          }
        }
        q.appendAll(top.nexts.map(i=>nodes(i-1)))
    }
    result.zipWithIndex.foreach{case(r,i)=>
      println(s"the shortest distance to ${i+1} is $r")
    }
}
case class GraphNode(number:Int,nexts:Seq[Int])
solution()

ArrayBuffer(0, -1, -1, -1)
ArrayBuffer(0, 1, -1, -1)
ArrayBuffer(0, 1, -1, 1)
ArrayBuffer(0, 1, -1, 1)
ArrayBuffer(0, 1, 2, 1)
the shortest distance to 1 is 0
the shortest distance to 2 is 1
the shortest distance to 3 is 2
the shortest distance to 4 is 1


import scala.collection.mutable.{ArrayBuffer,Queue}

input: String = """1 2 2 4
2 1 4
3 0
4 1 3"""
nodes: Seq[GraphNode] = List(
  GraphNode(number = 1, nexts = List(2, 4)),
  GraphNode(number = 2, nexts = List(4)),
  GraphNode(number = 3, nexts = List()),
  GraphNode(number = 4, nexts = List(3))
)
defined function solution
defined class GraphNode

グラフの連結成分の個数を数える

- DFS,BFSでノードを探索する. DFSまたはBFSを行った回数=連結成分の個数

In [198]:
val nodes = Seq(GraphNode(1,Seq(2,4)),GraphNode(2,Seq(4)),GraphNode(3,Seq()),GraphNode(4,Seq()))
def solution()={
    var result= ArrayBuffer.fill(nodes.length)(-1)
    var cnt = 0
    for(i <- 0 to nodes.length-1){
        if(result(nodes(i).number-1) == -1){
            cnt+=1
            // 未探索のnodeからbfs を行う
            var q = Queue[GraphNode]()
            q.append(nodes(i))
            while(!q.isEmpty){
                // qから遷移可能な点をマークする
                val top = q.dequeue
                top.nexts.foreach{n=>
                  if(result(n-1) == -1){
                      result(n-1) = 1
                      q.append(nodes(n-1))
                  }
                }
                
            }
            
        }
    }
    cnt
}
solution()

nodes: Seq[GraphNode] = List(
  GraphNode(number = 1, nexts = List(2, 4)),
  GraphNode(number = 2, nexts = List(4)),
  GraphNode(number = 3, nexts = List()),
  GraphNode(number = 4, nexts = List())
)
defined function solution
res197_2: Int = 2

連結成分に含まれるかどうかを探索

- ノードをBFS,DFSで探索し、同じノードごとに連結成分の番号を振る.

In [214]:
import scala.collection.mutable.Map
//sorted graph nodes
val nodes = Seq(GraphNode(1,Seq(2,4)),GraphNode(2,Seq(4)),GraphNode(3,Seq()),GraphNode(4,Seq()))
def solution()={
    // init array
    var found = Array.fill(nodes.length)(-1)
    var graphId = 0
    var map = Map[Int,Int]() // nodeNumber(0-indexed) -> graphId
    for(i <- 0 to nodes.length-1){
        if(found(i)== -1){
            // start bfs
            graphId+=1 
            var q = Queue[GraphNode]()
            q.enqueue(nodes(i))
            found(i)==1
            map(i)=graphId

            while(!q.isEmpty){
                val top  = q.dequeue
                top.nexts.foreach{n=>
                  if(found(n-1)== -1){
                      found(n-1)= 1
                      q.enqueue(nodes(n-1))
                      map(n-1)=graphId
                  }
                }
            }
        }
    }
    println(map)
    map.values.toSet.size
}
solution()

HashMap(0 -> 1, 1 -> 1, 2 -> 2, 3 -> 1)


import scala.collection.mutable.Map
//sorted graph nodes

nodes: Seq[GraphNode] = List(
  GraphNode(number = 1, nexts = List(2, 4)),
  GraphNode(number = 2, nexts = List(4)),
  GraphNode(number = 3, nexts = List()),
  GraphNode(number = 4, nexts = List())
)
defined function solution
res213_3: Int = 2

木の直径

In [55]:
//https://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=GRL_5_A&lang=jp


def parseEdge(graph:Array[Array[Double]],input:String,separator:String=" ") :Unit={
    // int,int, (Int|Double)
    val edge = input.split(separator)
    val Array(from,to) = edge.take(2).map(_.toInt)
    val cost :Double = edge.last.toDouble
    graph(from)(to) = cost
    graph(to)(from) = cost
    graph(from)(from)=0
    graph(to)(to)=0
}
def parseEdges(n:Int,input:String) ={
    val graph = Array.fill(n,n)(-1.0)
    input.split("\n").foreach(edge=> parseEdge(graph,edge))
    graph
}


defined function parseEdge
defined function parseEdges

In [80]:

import scala.collection.mutable.Queue
val input:String= """0 1 2
1 2 1
1 3 3"""
val n=4
// graph(i)(j) i番目のノードからj番目のノードへのコスト. 存在しない部分は-1を入れる
// 無向グラフなので双方向
val graph =parseEdges(n,input)
val input2:String="""0 1 1
1 2 2
2 3 4"""
val graph2 = parseEdges(n,input2)

import scala.reflect.ClassTag

trait EdgeDistance[T] {
    def add(a:T,b:T):T
    def mzero:T
    def empty:T
}
implicit val intDistance = new EdgeDistance[Int]{
  def add(a: Int, b: Int): Int = a + b
  def mzero: Int = 0
  def empty:Int = -1
}
implicit val doubleDistance = new EdgeDistance[Double]{
    def add(a:Double,b:Double):Double = a + b
    def mzero :Double = 0.0
    def empty:Double = -1.0
}

// 二次元配列で表現された無向グラフが与えられたとき、ノードi から ノードj への距離を求める
// graph は graph(i)(j) がノードiからノードjへのコストをあらわす二次元配列である
// エッジがある場合ノードからノードへのコストを正の数で、ない場合-1であらわしたものを与える
// 負のコストは探索しない
def shortestPathInDAGByBFS[T](
    from:Int/*start node index*/,
    graph:Array[Array[T]])
(implicit comparator: Ordering[T],
 edgeDistance: EdgeDistance[T],ct:ClassTag[T]):(Int,T,Array[T])
/*(furthest node index,the furthest distance,distances)*/
= {
    // queue of indexes to search
    var q = Queue[Int]()
    // interim furthest node index and distance
    var furthest:(Int,T) = (from,edgeDistance.mzero)
    // distances between from node and `k`th node. (k=0,1,...,graph.length-1)
    var dist: Array[T]= Array.fill(graph.length)(edgeDistance.empty)
    
    dist(from)=edgeDistance.mzero
    
    q.enqueue(from)
    while(!q.isEmpty){
        val pwd = q.dequeue
        val nexts = graph(pwd)
        nexts.zipWithIndex.foreach{ case (d,idx)=>
            // d can be Int,Double,Float,BigInt
          if(comparator.compare(d,edgeDistance.mzero) >0){ // pwdからの距離
              if(dist(idx) == -1){// from から idx番目のノードへの距離が未探索
                    dist(idx) = edgeDistance.add(dist(pwd),d)
                   if(comparator.compare(dist(idx),furthest._2)> 0){
                       furthest = (idx,dist(idx))
                   }
                    q.append(idx)
              }
          }
        }
        
    }
    (furthest._1,furthest._2,dist)
}


def solution[T:EdgeDistance:ClassTag:Ordering](graph:Array[Array[T]])={
    val (i,d,ds)=shortestPathInDAGByBFS(0,graph)
    val (_,diameter,__) = shortestPathInDAGByBFS(i,graph)
   diameter     
}
solution[Double](graph)

solution[Double](graph)

import scala.collection.mutable.Queue

input: String = """0 1 2
1 2 1
1 3 3"""
n: Int = 4
graph: Array[Array[Double]] = Array(
  Array(0.0, 2.0, -1.0, -1.0),
  Array(2.0, 0.0, 1.0, 3.0),
  Array(-1.0, 1.0, 0.0, -1.0),
  Array(-1.0, 3.0, -1.0, 0.0)
)
input2: String = """0 1 1
1 2 2
2 3 4"""
graph2: Array[Array[Double]] = Array(
  Array(0.0, 1.0, -1.0, -1.0),
  Array(1.0, 0.0, 2.0, -1.0),
  Array(-1.0, 2.0, 0.0, 4.0),
  Array(-1.0, -1.0, 4.0, 0.0)
)
import scala.reflect.ClassTag


defined trait EdgeDistance
intDistance: AnyRef with EdgeDistance[Int] = ammonite.$sess.cmd79$Helper$$anon$1@fa45f22
doubleDistance: AnyRef with EdgeDistance[Double] = ammonite.$sess.cmd79$Helper$$anon$2@7a84442
defined function shortestPathInDAGByBFS
defined function solution
res79_12: Double = 5.0
res79_13: Double = 5.0

In [81]:
import $file.helpers.scala.{shortestPathInDAGByBFS=>DAGBFS}
import DAGBFS.doubleDistance
DAGBFS.shortestPathInDAGByBFS(0,graph)

import $file.$                                             

import DAGBFS.doubleDistance

res80_2: (Int, Double, Array[Double]) = (3, 5.0, Array(0.0, 2.0, 3.0, 5.0))

Container With Most Water

インデックスを$i \in \mathbb{N}(0 \in \mathbb{N})$として${a_0,a_1,\dots,a_i,\dots,a_{n-1}},(a_i \in \mathbb{N})$が与えられたとき, 各$a_i$について点$A_i=(i,a_i)$と点$R_i=(i,0)$を結ぶ線分$l_i$を描く. このとき、適当な$l_j$と$l_k(j\neq k,a_j<a_k)$を選び,高さ$a_j$,幅$|j-k|$によってつくられる長方形$S$を最大にするとき、その最大値を求めよ.

解の方針

二つのインデックスを両側から順に動かしていく.内側に動かすとき高さは増加・減少しうる一方で幅は減少することに注意する.


In [8]:
val heights= Seq(1,8,6,2,5,4,8,3,7)

heights: Seq[Int] = List(1, 8, 6, 2, 5, 4, 8, 3, 7)

In [9]:
def solution(heights:Seq[Int],leftEnd:Int=0,rightEnd:Int,max:Int):Int = {
    var _l = leftEnd
    var _r = rightEnd
    var _max = max
    if(_l < _r){
        val S = (_r - _l)* math.min(heights(_l),heights(_r))
       _max = math.max(max,S)
        if(heights(_l)>heights(_r)){
            _r -= 1
        }else{
            _l +=1
        }
    solution(heights,_l,_r,_max)
    }else{
        _max
    }
}

defined function solution

In [11]:
solution(heights,0,heights.length-1,0)

res10: Int = 49

In [86]:
import almond.interpreter.api.DisplayData
display(
  DisplayData(
    Map(
      "text/html" -> "Some <b>HTML</b>","text/md"->"## hello world")))

Some HTML

import almond.interpreter.api.DisplayData


RomanToInteger

note:
- HashMapの要素アクセスは$O(1)$

In [12]:
def RomanToInteger(roman:String):Int={
    var sum :Int = 0
    val pairs:Map[String,Int] = Map("I"->1,"V"->5,"X"->10,"L"->50,"C"->100,"D"->500,"M"->1000)
    for(i <- 0 to roman.length -2){
        if(pairs(roman(i).toString)<pairs(roman(i+1).toString)){
          sum -= pairs(roman(0).toString)
        }else {
          sum += pairs(roman(i).toString)
        }
        
    }
    return sum + pairs(roman.last.toString)
}

defined function RomanToInteger

3Sum
- $\alpha + \beta + \gamma = 0$なら、どれか一つを固定すればTwoSumに帰着できる.
- sortすれば大きすぎる値は除けそう

In [45]:
import scala.collection.mutable.ArrayBuffer
import scala.collection.mutable.HashMap
import scala.collection.mutable.Set
val nums = Seq(-1,0,1,2,-1,-4).sorted
// a->c->bのidx
val map:HashMap[Int,HashMap[Int,Int]]=HashMap()
val set:Set[Seq[Int]] = Set()
for(i <- 0 to nums.length -1){
    val fixed = nums(i)
    if(!map.contains(fixed)){
        map.addOne(fixed->HashMap())
        for (j <- 0 to nums.length -1){
             if(j!=i){
              if(map(fixed).contains(nums(j))){
                set.add(Seq(fixed,nums(j),nums(map(fixed)(nums(j)))).sorted)
            }else{
              map(fixed).addOne(-fixed-nums(j) -> j)
            }
        }     
    } 
    }     
}
set

import scala.collection.mutable.ArrayBuffer

import scala.collection.mutable.HashMap

import scala.collection.mutable.Set

nums: Seq[Int] = List(-4, -1, -1, 0, 1, 2)
map: HashMap[Int, HashMap[Int, Int]] = HashMap(
  -1 -> HashMap(1 -> 3, 2 -> 2, 5 -> 0),
  0 -> HashMap(-2 -> 5, 1 -> 2, 4 -> 0),
  1 -> HashMap(0 -> 2, -3 -> 5, 3 -> 0),
  2 -> HashMap(-1 -> 1, -2 -> 3, -3 -> 4, 2 -> 0),
  -4 -> HashMap(2 -> 5, 3 -> 4, 4 -> 3, 5 -> 2)
)
set: Set[Seq[Int]] = HashSet(List(-1, 0, 1), List(-1, -1, 2))
res44_7: Set[Seq[Int]] = HashSet(List(-1, 0, 1), List(-1, -1, 2))

方針は探索と枝かり

> Given a string s containing only digits, return all possible valid IP addresses that can be obtained from s. You can return them in any order.

>A valid IP address consists of exactly four integers, each integer is between 0 and 255, separated by single dots and cannot have leading zeros. For example, "0.1.2.201" and "192.168.1.1" are valid IP addresses and "0.011.255.245", "192.168.1.312" and "192.168@1.1" are invalid IP addresses. 

 

In [113]:
import scala.collection.mutable.Set

def validIp(str:String,results:Set[String],availableDots:Int=3,tmp:String=""):Unit={
    if(availableDots==0){
        results.add(tmp+str)
    }else{
        divide(str).foreach{case (h,t)=>
        if(isValidIpComponent(h)&&isValidIpTrailing(t,availableDots-1)){
            validIp(t,results,availableDots-1,tmp+h+".")
        }
      
      }
    }   
}
def isValidIpComponent(str:String):Boolean={
    if(str.toInt > 255){
        false
    }else{
        true
    }
    
}
def isValidIpTrailing(str:String,availableDots:Int):Boolean={
    if(str.length < availableDots ){
        false
    }else if(availableDots<=0 & (str.length>3 || str.length>1 & str.startsWith("0"))){
        false
    }else if(str.length==0){
        false
    }else{
        true
    }
}
def divide(str:String):Seq[(String,String)]={
    if(str.startsWith("0")){
        Seq(("0",str.substring(1)))
    }else{
        (1 to math.min(str.length,3)).map { i =>
           (str.substring(0,i),str.substring(i))
        }.toSeq 
    }
}

import scala.collection.mutable.Set


defined function validIp
defined function isValidIpComponent
defined function isValidIpTrailing
defined function divide

上の解答では再帰を使って状態を受け渡すように書いているが、やっていることは深さの最大が3の深さ優先探索と同じ.

In [114]:
var results = Set[String]()
validIp(str="1111",results)
print(results)

HashSet(1.1.1.1)

results: Set[String] = HashSet("1.1.1.1")

In [115]:
import scala.collection.mutable.Stack
def dfs(str:String)={
    var s = Stack[String]()
    var result = Set[String]()
    val availableDots=3
    s.push("")
    validIpWithStack(str,availableDots,s,result)
    print(result)
}
def validIpWithStack(str:String,availableDots:Int,s:Stack[String],result:Set[String]):Unit={
    if(s.isEmpty){
        result
    }else{
        if(availableDots==0){
            var tmp= s.pop()
            print(str)
            if(isValidIpTrailing(str,0)){
                result.add(tmp+str)     
            }
        }else{
        var tmp = s.pop
        var nexts = getNexts(str)
        for(j <- 0 to nexts.length -1){
            val tail = str.substring(nexts(j).length)
            if(isValidIpComponent(nexts(j)) && isValidIpTrailing(tail,availableDots-1)){
                 s.push(tmp+nexts(j)+".")
                 validIpWithStack(tail,availableDots-1,s,result)
                
        
      
            }
            
        }  
        }
         
    }
    
}
def getNexts(str:String):Array[String]={
        if(str.startsWith("0")){
            Array("0")
        }else{
        (1 to math.min(str.length,3)).map { i =>
           str.substring(0,i)
        }.toArray
    }
 }

import scala.collection.mutable.Stack

defined function dfs
defined function validIpWithStack
defined function getNexts

In [112]:
dfs("010010")

100HashSet(0.100.1.0, 0.10.0.10)

In [2]:
val in="5 2\n3 1\n3 1\n10 3\n1 10\n10 1\n8 3\n0 0\n".stripMargin

def parse(input:String,state:Seq[(Int,Seq[(Int,Int)])]=Seq()):Seq[(Int,Seq[(Int,Int)])] = {
    val header = input.take(input.indexOf("\n")).split(" ").map(_.toInt)
    if(header(0)== 0 && header(1)==0){
        return state
    }
    parse(next(input,header(1)+1),state.appended(aggregate(input,header)))  
}
def next(input:String,skips:Int):String ={
    if(skips==0){
        return input
    }else{
        next(input.drop(input.indexOf("\n")+1),skips-1)  
    }
}

def readlines(input:String,lines:Int,state:StringBuilder=new StringBuilder()):String = {
    if(lines==0){
        return state.toString
    }else if(input.indexOf("\n")== -1){
        readlines("",lines-1,state.append(input))
    }else{
        val l = input.substring(0,input.indexOf("\n")+1)
        readlines(input.drop(l.length),lines-1,state.append(l))
    }
}

def aggregate(input:String,header:Array[Int]):(Int,Seq[(Int,Int)])={
    (header(0),readBody(next(input,1),header(1)))
    
}
def readBody(str:String,rows:Int):Seq[(Int,Int)]={
    readlines(str,rows).split("\n").map{line => val splited = line.split(" ").map(_.toInt);(splited(0),splited(1))}
}
val datasets = parse(in)

in: String = """5 2
3 1
3 1
10 3
1 10
10 1
8 3
0 0
"""
defined function parse
defined function next
defined function readlines
defined function aggregate
defined function readBody
datasets: Seq[(Int, Seq[(Int, Int)])] = List(
  (5, ArraySeq((3, 1), (3, 1))),
  (10, ArraySeq((1, 10), (10, 1), (8, 3)))
)

In [58]:
def action(arr:Array[Int],ops:(Int,Int))={
    val p = ops._1
    val c = ops._2
    val n = arr.length
    val up = arr.slice(n-p-c+1,n-p+1)
    for(i <- n-p-c+1 to n-1){
        if(i<=n-p+1 || i+c < n){
            if(i+c < n){
                arr(i)=arr(i+c)
            }
        }else{
            
            arr(i)=up(i-n+c)  
        }
    }         
}
datasets.foreach{ case (n,operations)=>
    val arr = Array.range(1,n+1)
  operations.foreach{case (p,c)=>
    action(arr,(p,c))
  }
    println(arr.last)
}


4
4


defined function action

# 

In [168]:

def phoneButton(
    input:String,
    result:StringBuilder=new StringBuilder(),
    keyMap:Map[String,Array[String]]= Map(
        "1"->Array(".",",","!","?"," "),
        "2"->Array("a","b","c"),
        "3"->Array("d","e","f"),
        "4"->Array("g","h","i"),
        "5"-> Array("j","k","l"),
        "6"->Array("m","n","o"),
        "7"->Array("p","q","r","s"),
        "8"->Array("t","u","v"),
        "9"->Array("w","x","y","z")
    )):String={
    if(input.length==0){
        return result.toString
      }
    val enter = input.dropWhile(_.toString=="0").takeWhile(_.toString!="0")
   
    if(enter.isEmpty){
        return result.toString
    }
    
    enter.head.toString match {
        case n @ ("2"|"3"|"4"|"5"|"6"|"8") =>
          phoneButton(
              nextInput(input),
              result.append(
                        keyMap(n).apply(((enter.length+2)%3))
          ),keyMap)
        case n @("7"|"9") =>
            phoneButton(
              nextInput(input),
                result.append(
                    keyMap(n).apply(((enter.length+3)%4))
          ))
        case n if n== "1"=>
            phoneButton(
              nextInput(input),
                result.append(
                    keyMap(n).apply(((enter.length+4)%5))
            ),keyMap)
        case _ => phoneButton(nextInput(input),result,keyMap)
    }
}
def nextInput(input:String):String={
     input.dropWhile(_.toString=="0").dropWhile(_.toString!="0").dropWhile(_.toString=="0")
}

defined function phoneButton
defined function nextInput

In [169]:
phoneButton("44033055505550666011011111090666077705550301110")

res168: String = "hello, world!"

In [ ]:
import scala.collection.mutable.Map
val input= """3 2
2 1 4
0
3 3 4 8
3 2
4 1 5 8 9
3 2 5 9
5 2 4 5 7 9
3 3
2 1 4
3 2 5 9
2 2 4
3 3
2 1 2
3 1 2 9
2 2 4
0 0""".stripMargin

def parse(input:String):Seq[Int]={
    if(input.isEmpty){
        Seq()
    }else{
        val header = readlines(input,1).stripLineEnd.split(" ").map(_.toString.toInt)
        if(header(0)==0 &header(1)==0){
            return Seq()
        }
        val tail = next(input,1)
        val schedules = getSchedules(tail,header(0))
        getBestDay(schedules,header(1)) +: parse(next(input,header(0)+1))    
    }
    
}

def getSchedules(str:String,lines:Int):Seq[Set[Int]]={
    if(lines==0) Seq()
    else parseOneSchedule(readlines(str,1)) +: getSchedules(next(str,1),lines-1)
}
def parseOneSchedule(str:String):Set[Int]={
    str.stripLineEnd.split(" ").map(_.toString.toInt).drop(1).toSet
}
def getBestDay(schedules:Seq[Set[Int]],required:Int):Int={
    var candidates: Map[Int,Int]=Map()
    schedules.foreach{schedule=>
        schedule.foreach{day=>
          if(candidates.contains(day)){
              candidates(day)+=1
              if(candidates(day)>=required){
                  return day
              }
          }else{
              candidates(day)=1
          }
        }
    }
    0
}
parse(input)

In [173]:
def search(price:Int,purse:Seq[Int])={
    var min = (80,20,20,20,20)
    // minimize purse.sum() - (k+l+m+n)+change
    // where 0<=k<=purse(1),0<=l<=purse(1),0<=m<=purse(2),0<=n<=purse(3)
    // and 10*k + 50*l+100*m+500*n>=price
    for(k <- 0 to purse(0)){
        for(l <- 0 to purse(1)){
            for(m <- 0 to purse(2)){
                for(n <- 0 to purse(3)){
                    val pay = 10*k+50*l+100*m+500*n
                    if(pay>=price){
                        val cnt = purse.sum - k -l - m - n + optimalChange(price,pay)
                        if(min._1 > cnt){
                            min = (cnt,k,l,m,n)
                        }
                    }
                }
            }
        }
        
    }
    
    min
}
def optimalChange(price:Int,pay:Int)={
    val change = pay- price
    val n = change / 500
    val m = (change -500*n) /100
    val l = (change - 500*n - 100*m)/50
    val k= (change - 500*n - 100*m -50*l)/10
    k+l+m+n
}


defined function search
defined function optimalChange

全探索を使って解いたが、問題を整理するともっと効率よく解を求められる.

解を$o^*$,purseの中の枚数合計を$S$,おつりの枚数を$c$とすると$o^*=minimize\{ S-(k+l+m+n) + c\}$となる. 

ここで最適なおつりの枚数を$c^*$とすると$k,l,m,n$について$c^*=f(k,l,m,n)=minimize \{ o - S + (k+l+m+n) \}$となる. 

この式からてもちの$k,l,m,n$をすべて支払ったときの最適なおつり$c^*$は最適な支払い$o^*$と等しいと考えられる.

In [49]:
val input = """6 9
....#.
.....#
......
......
......
......
......
#@...#
.#..#.
11 9
.#.........
.#.#######.
.#.#.....#.
.#.#.###.#.
.#.#..@#.#.
.#.#####.#.
.#.......#.
.#########.
...........
11 6
..#..#..#..
..#..#..#..
..#..#..###
..#..#..#@.
..#..#..#..
..#..#..#..
7 7
..#.#..
..#.#..
###.###
...@...
###.###
..#.#..
..#.#..
0 0"""


input: String = """6 9
....#.
.....#
......
......
......
......
......
#@...#
.#..#.
11 9
.#.........
.#.#######.
.#.#.....#.
.#.#.###.#.
.#.#..@#.#.
.#.#####.#.
.#.......#.
.#########.
...........
11 6
..#..#..#..
..#..#..#..
..#..#..###
..#..#..#@.
..#..#..#..
..#..#..#..
7 7
..#.#..
..#.#..
###.###
...@...
###.###
..#.#..
..#.#..
0 0"""

In [50]:
import scala.collection.mutable.{Queue,Set}
def mazeParser(str:String,result:Seq[Int]=Seq()):Seq[Int]={
    val header = readlines(str,1).stripLineEnd.split(" ").map(_.toString.toInt)
    if(header(0)==0 & header(1)==(0)){
        return result
    }
    val body = readlines(next(str,1),header(1))
    val tiles = convert(body,header(0)) match {
        case (startFrom,maze:Seq[String])=> run(maze,startFrom,(header(1),header(0)))
    }
    mazeParser(next(str,header(1)+1),result.appended(tiles))
    
}
def convert(str:String,colLength:Int):((Int,Int),Seq[String]) = {
    val rle = str.replace("\n","")
   ((rle.indexOf("@")/colLength,(rle.replace("\n","").indexOf("@"))%colLength),str.split("\n"))
}
def run(maze:Seq[String],startFrom:(Int,Int),dimension:(Int,Int))= {
    var q = Queue[(Int,Int)]()
    q.enqueue(startFrom)
    var history = Set[(Int,Int)](startFrom)
    while(!q.isEmpty){
        val visited = q.dequeue
        // (y,x) -> next: (y-1,x),(y,x-1),(y+1,x),(y,x+1)
        // that history does not contains and is not # 
        Set(
            (math.max(0,visited._1-1),visited._2),
            (visited._1,math.max(0,visited._2-1)),
            (math.min(dimension._1-1,visited._1+1),visited._2),
            (visited._1,math.min(dimension._2-1,visited._2+1))).foreach {
            case position @ (y_next,x_next)=>
              if(!history.contains(position) & maze(y_next)(x_next).toString!="#"){
                history.add(position)
                q.enqueue(position)
            }
   
        }
    }
    history.size
}
mazeParser(input)

import scala.collection.mutable.{Queue,Set}

defined function mazeParser
defined function convert
defined function run
res49_4: Seq[Int] = List(45, 59, 6, 13)

In [16]:
val input ="""2
10 11
11 12
2
N 2
E 1
2
10 11
11 12
2
N 2
W 1
3
0 15
5 10
5 15
5
W 10
S 10
N 20
E 10
S 10
0
"""

input: String = """2
10 11
11 12
2
N 2
E 1
2
10 11
11 12
2
N 2
W 1
3
0 15
5 10
5 15
5
W 10
S 10
N 20
E 10
S 10
0
"""

In [48]:
def read(input:String)= {
    // split input
    val arr= input.split("\n")
    // print results
    println(solve(arr).toString)
    
}
def solve(arr:Seq[String],results:StringBuilder=new StringBuilder()) :String= {
        if(arr.isEmpty){
            return results.toString
        }else{
     getPosition(arr) match {
        case (operations,positions) if(!positions.isEmpty)=>
          getMoves(operations) match {
              case (next,moves)=> 
                val result = run(moves,positions,positions.size)
                solve(next,results.append(result+"\n"))
          }
        case  _ => return results.toString
      }
             
    }

    
}
def getPosition(input:Seq[String]):(Seq[String],scala.collection.immutable.Set[(Int,Int)]) ={
    // read the first line of input to get N
    val N = input(0).toString.toInt
    if(N==0){
        return (input.drop(1),scala.collection.immutable.Set.empty)
    }
    val positions = input.drop(1).take(N).map(_.stripLineEnd.split(" ").map(_.toString.toInt)).map(a=>(a(0),a(1))).toSet
    (input.drop(N+1),positions)
    // get next N lines as an array of (Int,Int)
    // return  input.drop(N+1),positions
}
def getMoves(input:Seq[String])= {
    // read the first line of input to get M
    val M = input(0).toString.toInt
    // get next M ines as array of (String,Int)
    val moves = input.drop(1).take(M).map(_.stripLineEnd.split(" ").map(_.toString)).map(b =>   (  b(0).toString,b(1).toInt ) )
     // return input.drop(M+1),moves
    (input.drop(M+1),moves)
}

def run(moves:Seq[(String,Int)],gems:scala.collection.immutable.Set[(Int,Int)],remains:Int,startAt:(Int,Int)=(10,10)):String={
    if(remains==0){
        return "yes"
    }else if(moves.isEmpty & remains>0){
        return "no"
        
    }else {
        val move = moves.head
        getDiff(move) match {
            case  ( dx, dy) if gems.exists(position=>check(startAt,(startAt._1+dx,startAt._2+dy),position)) =>
               run(moves.drop(1),gems -((startAt._1+ dx,startAt._2+ dy)),remains-1,(startAt._1+ dx,startAt._2+ dy))
            case (dx,dy) => run(moves.drop(1),gems,remains,(startAt._1+ dx,startAt._2+dy))
        }
        
    }
}

def getDiff(move:(/*direction*/String,/*distance*/Int)):/*(dx,dy):*/(Int,Int)= {
    move match {
        case ("N",distance) => (0,distance)
        case ("W",distance) => (-distance,0)
        case ("E",distance)=> (distance,0)
        case ("S",distance) => (0,-distance)
    }
}
//  -> x == gem_x & (y <) gem_y <= y_next  || y==gem_y & (x < )gem_x <= x_next
def check(from:(Int,Int),to:(Int,Int),gemPosition:(Int,Int)):Boolean  ={
    (from,to,gemPosition) match {
        case ((x,y),(x_next,y_next),(gem_x,gem_y)) if (( y < gem_y & x==gem_x & gem_y <= y_next) || (y==gem_y & x < gem_x & gem_x <= x_next)  ) =>
          true
        case _ => false
    }
}
read(input)

yes
no
no



defined function read
defined function solve
defined function getPosition
defined function getMoves
defined function run
defined function getDiff
defined function check

In [18]:
import scala.collection.mutable.ArrayBuffer
val input ="""4 4 1
3 1 2
2 2 3
3 3 4
1 3 4
0 0 0
"""
implicit val nodeOrdering = new Ordering[Node]{
    def compare(left:Node,right:Node):Int =
        (left.height,right.height) match {
            case (h_1,h_2) if h_1 > h_2 => -1
            case (h_1,h_2) if h_1 == h_2 => 0
            case _ => 1
        }
}
def solve(input:String)(implicit comparator: Ordering[Node]):Unit={
    val arr = input.split("\n").map(_.split(" ").map(_.toString.toInt))
    val n = arr(0)(0)
    val m = arr(0)(1)
    val a = arr(0)(2)
    (n,m,a) match {
        case (0,0,0)=>return
        case (n,m,a) =>
            var lines : Array[ArrayBuffer[Node]]= Array.fill(n)(ArrayBuffer.empty[Node])
            parse(arr.drop(1),lines)
            val amida =lines.map(line=>line.sorted(comparator).toArray)
            val end = run(amida,a-1)
            println(end+1)
    }
    
}
def parse(arr:Array[Array[Int]],lines:Array[ArrayBuffer[Node]])={
    arr.foreach{row =>
        // row(1)-1,row(2) -1 番目の行に 
        //Node(height=row(0),left=None,right=Some(row(2)-1)),
        //Node(height=row(0),Some(row(1)-1),right=None) を追加する
        lines(math.max(0,row(1)-1)) +=(Node(height=row(0),left=None,right=Some(math.max(0,row(2)-1))))
        lines(math.max(0,row(2)-1)) +=(Node(height=row(0),left=Some(math.max(0,row(1)-1)),right=None))    
    }
    
}
// 高さcurrentHeight 以下のあみだくじlinesをlineからスタートして解く.
def run(lines:Array[Array[Node]],line:Int,currentHeight:Int=1001):Int={
    if(lines(line).isEmpty) return line
    lines(line).head match {
        case Node(height,Some(nextLine),None)  => run(lines.map(l=>l.dropWhile{case Node(h,_,_)=>h >=height}),nextLine,height-1)
        case Node(height,None,Some(nextLine)) => run(lines.map(l=>l.dropWhile{case Node(h,_,_)=>h >=height}),nextLine,height-1)
    }
    
    
}

final case class Node(height:Int,left:Option[Int],right:Option[Int])
solve(input)

4


import scala.collection.mutable.ArrayBuffer

input: String = """4 4 1
3 1 2
2 2 3
3 3 4
1 3 4
0 0 0
"""
nodeOrdering: Object with Ordering[Node] = ammonite.$sess.cmd17$Helper$$anon$1@e30a4ab
defined function solve
defined function parse
defined function run
defined class Node

In [17]:
val data = Seq(1,2,4,27,300,1250)
data.foreach{e =>
    var m = e
    for( z <- 0 to math.pow(e,1.0/3.0).toInt){
        for(y <- 0 to math.pow(e,0.5).toInt){
            var next = math.min(m,math.max(0,e+ y + z - y*y  - z*z*z))
            if(next-y-z>=0){
               m =  next
        
            }
        }
    }
    println(m)
}

1
2
2
3
18
44


data: Seq[Int] = List(1, 2, 4, 27, 300, 1250)

In [30]:
val input1 = """05:47:15 09:54:40
12:12:59 12:13:00
16:30:20 21:18:53"""
val input2 = """00:00:00 03:00:00
01:00:00 03:00:00
03:00:00 05:00:00
02:00:00 03:00:00
03:00:00 04:00:00
03:00:00 06:00:00"""

def solution(input:String)={
    val schedules = input.split("\n").map(_.split(" ")).sortBy(schedule=>schedule(0))
    var trains :ArrayBuffer[String]= ArrayBuffer("00:00:00")
    schedules.foreach {schedule =>
        val leaveAt = schedule(0)
        val arriveAt = schedule(1)
        trains.indexWhere(_ <= leaveAt) match {
            case -1 => trains.append(arriveAt)
            case idx =>  trains(idx)=arriveAt
        }  
    }
    trains.length
}
solution(input2)

input1: String = """05:47:15 09:54:40
12:12:59 12:13:00
16:30:20 21:18:53"""
input2: String = """00:00:00 03:00:00
01:00:00 03:00:00
03:00:00 05:00:00
02:00:00 03:00:00
03:00:00 04:00:00
03:00:00 06:00:00"""
defined function solution
res29_3: Int = 3

In [44]:

// zigzag conversion
// 例
// 与えられる文字列: "PAYPALISHIRING"
// numRows: 3
// 想定される並び方
//P   A   H   N
//A P L S I I G
//Y   I   R
//numRows: 4
//P     I    N
//A   L S  I G
//Y A   H R
//P     I
// numRows+numsRows-2の周期なのでindex % (numRows + (numRows-2))について考える
def zigzagConversion(input:String,rowNums:Int):String = {
    var arr:Array[StringBuilder]= Array.fill(rowNums)(new StringBuilder())
    for(i <- 0 to input.length-1){
        if(i%(rowNums*2-2)<=rowNums-1){
            arr(i % (rowNums+ rowNums -2)).append(input(i))  
        }else{
            arr( rowNums*2-2-(i % (rowNums+ rowNums -2))).append(input(i))
        }
    }
    arr.map(_.toString).mkString
    
    
}
val input = "PAYPALISHIRING"
zigzagConversion(input,3)=="PAHNAPLSIIGYIR"
zigzagConversion(input,4)=="PINALSIGYAHRPI"

defined function zigzagConversion
input: String = "PAYPALISHIRING"
res43_2: Boolean = true
res43_3: Boolean = true

In [48]:
import scala.collection.mutable.ArrayBuffer
val input = """5
5
0 0
2 0
2 1
4 1
4 0
5
0 0
0 2
-1 2
-1 4
0 4
5
0 0
0 1
-2 1
-2 2
0 2
5
0 0
0 -1
2 -1
2 0
4 0
5
0 0
2 0
2 -1
4 -1
4 0
5
0 0
2 0
2 1
4 1
4 0
4
4
-60 -75
-60 -78
-42 -78
-42 -6
4
10 3
10 7
-4 7
-4 40
4
-74 66
-74 63
-92 63
-92 135
4
-12 22
-12 25
-30 25
-30 -47
4
12 -22
12 -25
30 -25
30 47
3
5
-8 5
-8 2
0 2
0 4
8 4
5
-3 -1
0 -1
0 7
-2 7
-2 16
5
-1 6
-1 3
7 3
7 5
16 5
5
0 1
0 -2
8 -2
8 0
17 0
0"""

def splitInput(input:String):Array[String]={
    input.split("\n")
}
def readInput(
    arr:Array[String],
    from:Int=0,
    result:ArrayBuffer[Array[Array[(Int,Int)]]]=ArrayBuffer(),
    readFrom:Int=0):Array[Array[Array[(Int,Int)]]]={
    val n = arr(from).toString.toInt
    if(n==0){
        return result.toArray
    }else{
        val chunks = readChunks(arr,n+1,from)
        result.append(chunks)
        readInput(arr,from+chunks.map(_.length+1).sum+1,result)
    }
}
def readChunk(arr:Array[String],chunkFrom:Int=0):Array[(Int,Int)] = {
    // header
    val m = arr(chunkFrom).toString.toInt
    var result = ArrayBuffer[(Int,Int)]()
    for(i <- chunkFrom+1 to chunkFrom+m){
        val position = arr(i).split(" ").map(_.toString.toInt)
        result.append(  (position(0),position(1)) )
    } 
    result.toArray
}

def readChunks(
    arr:Array[String],
    n:Int,startFrom:Int=0,
    result:ArrayBuffer[Array[(Int,Int)]]=ArrayBuffer()):Array[Array[(Int,Int)]]={
    if(n==0){
        return result.toArray
    }else{
        val chunk = readChunk(arr,startFrom+1)
        result.append(chunk)
        readChunks(arr,n-1,startFrom+1+chunk.length,result)
    }
}
val chunks="""5
5
0 0
2 0
2 1
4 1
4 0
5
0 0
0 2
-1 2
-1 4
0 4
5
0 0
0 1
-2 1
-2 2
0 2
5
0 0
0 -1
2 -1
2 0
4 0
5
0 0
2 0
2 -1
4 -1
4 0
5
0 0
2 0
2 1
4 1
4 0
0""".split("\n")
val angles=Set(0,math.Pi/2.0,math.Pi,math.Pi*1.5)
val dataset = readInput(input.split("\n"))
  
for (i <- 0 to dataset.length-1){
    var lines = dataset(i)
    val original_line = lines(0)
    var A = (original_line(0)._1,original_line(0)._2)
//  l_0を原点に移動したもの
    var slided_by_OA = original_line.map{pos=>
        (pos._1 - A._1,pos._2 - A._2)
    }
    // l_0を除くl_i を始点を基準に原点に平行移動したもの
    val slided_by_start = lines.drop(1).map{line=>
       line.map(pos=>(pos._1 - line(0)._1,pos._2-line(0)._2) )                                     
    }
    // l_0を除くl_i を終点を基準に原点に平行移動したもの
    val slided_by_end = lines.drop(1).map{line=>
      line.map(pos => (pos._1 - line(line.length-1)._1,pos._2-line(line.length-1)._2))
    }
    
    slided_by_start.zipWithIndex.foreach { case (line,lineIdx)=>
        angles.foreach{theta =>
            val rotated = line.map(pos => (
                      (math.cos(theta)*pos._1-math.sin(theta)*pos._2).round.toInt,
                      (math.sin(theta)*pos._1 + math.cos(theta)*pos._2).round.toInt
                  ))
            if(rotated.zipWithIndex.forall{case (pos,idx)=>
                pos == slided_by_OA(idx)
            }) {
                
                println(s"same at ${lineIdx+1} with rotation $theta")
                
            }       }
        
    }
    slided_by_end.zipWithIndex.foreach { case (line,lineIdx)=>
        angles.foreach{theta =>
            val rotated = line.map(pos => (
                      (math.cos(theta)*pos._1-math.sin(theta)*pos._2).round.toInt,
                      (math.sin(theta)*pos._1 + math.cos(theta)*pos._2).round.toInt
                  ))
            if(rotated.zipWithIndex.forall{case (pos,idx)=>
                pos == slided_by_OA(slided_by_OA.length-idx-1)
            }) {
                
                println(s"same at ${lineIdx+1} with rotation $theta")
                
            }       }
        
    }
    
    println("+++++")
}

same at 1 with rotation 4.71238898038469
same at 5 with rotation 0.0
same at 3 with rotation 3.141592653589793
+++++
same at 3 with rotation 3.141592653589793
same at 4 with rotation 0.0
+++++
+++++


import scala.collection.mutable.ArrayBuffer

input: String = """5
5
0 0
2 0
2 1
4 1
4 0
5
0 0
0 2
-1 2
-1 4
0 4
5
0 0
0 1
-2 1
-2 2
0 2
5
0 0
0 -1
2 -1
2 0
4 0
5
0 0
2 0
2 -1
4 -1
4 0
5
0 0
2 0
2 1
4 1
4 0
4
4
...
defined function splitInput
defined function readInput
defined function readChunk
defined function readChunks
chunks: Array[String] = Array(
  "5",
  "5",
  "0 0",
  "2 0",
  "2 1",
  "4 1",
  "4 0",
  "5",
  "0 0",
  "0 2",
  "-1 2",
  "-1 4",
  "0 4",
  "5",
  "0 0",
  "0 1",
  "-2 1",
  "-2 2",
  "0 2",
  "5",
  "0 0",
  "0 -1",
  "2 -1",
  "2 0",
  "4 0",
  "5",
  "0 0",
  "2 0",
  "2 -1",
  "4 -1",
  "4 0",
  "5",
  "0 0",
  "2 0",
  "2 1",
  "4 1",
  "4 0",
  "0"
)
angles: Set[Double] = Set(
  0.0,
  1.5707963267948966,
  3.141592653589793,
  4.71238898038469
)
dataset: Array[Array[Array[(Int, Int)]]] = Array(
  Array(
    Array((0, 0), (2, 0), (2, 1), (4, 1), (4, 0)),
    Array((0, 0), (0, 2), (-1, 2), (-1, 4), (0, 4)),
    Array((0, 0), (0, 1), (-2, 1), (-2, 2), (0, 2

In [28]:
val input = Array(0,1,0,11,5,1,1,0)

input: Array[Int] = Array(0, 1, 0, 11, 5, 1, 1, 0)

In [29]:
def moveZeros(arr:Array[Int],readFrom:Int=0):Array[Int]={
    if(arr.length-1 < readFrom){
        return arr
    }else{
        if(arr(readFrom)!=0){
            moveZeros(arr,readFrom+1)
        }else{
            var i = 0
            while(readFrom+i<arr.length-1 & arr(readFrom+i)==0){
                i+=1
            }
            arr(readFrom) = arr(readFrom+i)
            arr(readFrom+i)=0
            moveZeros(arr,readFrom+1)
        }
    }

}
moveZeros(input)

defined function moveZeros
res28_1: Array[Int] = Array(1, 11, 5, 1, 1, 0, 0, 0)

leetcode 50

> Implement pow(x, n), which calculates x raised to the power n (i.e., xn).



In [99]:
def pow(x:Double,n:Int,result:Double=1):Double={
    if(n==0){
        result
    }else if (n<0){
        pow(x,n+1,result/x)
    }else {
        pow(x,n-1,result*x)
    }
}

defined function pow

In [103]:
def efficientPow(x:Double,b:String,result:Double=1,i:Int=0):Double={
      if(b.length-1==i){
        if(b(b.length-1-i).toString=="1"){
            result*pow(x,pow(2,i).toInt)
        }else{
            result
        }
    }else{
        efficientPow(
            x,
            b,
            result*efficientPow(x,(b(b.length-1-i).toString.toInt*efficientPow(2,i.toBinaryString)).toInt.toBinaryString),
            i+1
        )
    }
}

defined function efficientPow
res102_1: Double = 1024.0

In [23]:
//  (a,d,n) a から始まり d ずつ増える 等差数列に含まれる素数のうちで n 番目のものを出力する
val input = Array(367,186,151)

input: Array[Int] = Array(367, 186, 151)

In [12]:
def seq(a:Int,d:Int,k:Int):Int={
    a+d*(k-1)
}
def solution(a:Int,d:Int,n:Int)= {
    // seq(a,d,k)に含まれるn番目の素数を求める
    // 素数のカウントがnになるまでkをincrementし続ける?
    var primeCount = 0
    var k = 1
    while(primeCount<n){
        if(isPrime(seq(a,d,k))){
            primeCount+=1
        }
        k+=1
    }
    seq(a,d,k-1)
    
}
def isPrime(x:Int):Boolean={
    if(x==2){
        true
    }else if(x%2==0){
        false
    }else{
      Range(3,math.sqrt(x).round.toInt+1,2).foreach{e=>
        if(x%e==0) return false
      }
     true  
    }
}
solution(179,10,203)

defined function seq
defined function solution
defined function isPrime
res11_3: Int = 6709

In [18]:
import scala.collection.mutable.ArrayBuffer

import scala.collection.mutable.ArrayBuffer

In [9]:
def solution(n:Int,k:Int)={
    row(n)(k-1)
}
def row(n:Int,prev:String="0"):String={
    if(n==1){
        return prev
    }else{
        row(n-1,convert(prev))
    }
}
def convert(s:String):String={
    var sb = new StringBuilder()
    for(i <- 0 to s.length-1){
        if(s(i).toString=="0"){
            sb.append("01")
        }else{
            sb.append("10")
        }
    }
    sb.toString
}

defined function solution
defined function row
defined function convert
res8_3: Char = '0'

In [71]:

def permutation(original:Array[Int])={
    val ab = ArrayBuffer[ArrayBuffer[Int]]()
    for(i <- 0 to original.length -1){
        ab.append(ArrayBuffer(original(i)))
    }
    
    permutation_impl(original.toSet,original.length,ab)
}
def permutation_impl(original:Set[Int],n:Int,prevs:ArrayBuffer[ArrayBuffer[Int]]):ArrayBuffer[ArrayBuffer[Int]]={
    if(n==1){
        prevs
    }else{
        permutation_impl(original,n-1,next(original,prevs))
    }
}

def next(original:Set[Int],prevs:ArrayBuffer[ArrayBuffer[Int]])={
    val ab:ArrayBuffer[ArrayBuffer[Int]] = ArrayBuffer()
    for(i <- 0 to prevs.length-1){
        val prev = prevs(i).toSet
        val candidates = original.diff(prev) 
        candidates.foreach{l=>
          ab.append(prevs(i).appended(l))
        }
    }
    ab
}

defined function permutation
defined function permutation_impl
defined function next

In [73]:
// e.g. candidates = [2,3,6,7], target = 7
//39. Combination Sum

val candidates = Array(2,3,5)
val target = 8

def solution(candidates:Array[Int],target:Int)= {
    S(target,candidates.toSet).map(_.sorted).filterNot(_.isEmpty).toSet
}

   

def S(t:Int,candidates:Set[Int],result:ArrayBuffer[ArrayBuffer[Int]]=ArrayBuffer[ArrayBuffer[Int]]()):ArrayBuffer[ArrayBuffer[Int]]={
    if(t<0){
        result
    }else if(t==0){
        result
    }
    else{
        candidates.map{c=>
          if(t-c==0){
             S(t-c,candidates,result.appended(ArrayBuffer(c)))
          }else{
              S(t-c,candidates,result).filter{ab=> ab.sum+c==t}.map(_.appended(c))
          }
        }.foldLeft(ArrayBuffer(ArrayBuffer[Int]())){case (acc,ab)=>
            ab.foreach{a=>
            acc.append(a)}
            acc
        }
        
    }
}
solution(candidates,target)

candidates: Array[Int] = Array(2, 3, 5)
target: Int = 8
defined function solution
defined function S
res72_4: Set[ArrayBuffer[Int]] = Set(
  ArrayBuffer(2, 2, 2, 2),
  ArrayBuffer(2, 3, 3),
  ArrayBuffer(3, 5)
)

In [22]:
def isSubsequence(s:String,t:String):Boolean ={
    checkContains(s.split(""),t)
}
def checkContains(arr:Array[String],t:String,readFrom:Int=0,found:Int=0):Boolean={
    if(found==arr.length){
        true
    }else if(readFrom>t.length-1){
        false
    }else {
        if(t(readFrom).toString==arr(found)){
            checkContains(arr,t,readFrom+1,found+1)
        }else{
            checkContains(arr,t,readFrom+1,found)
        }   
    }
}

defined function isSubsequence
defined function checkContains

In [23]:
isSubsequence("abc","cacccbgdcd")

res22: Boolean = true

In [24]:

def countLevel(s:String,readFrom:Int=0,level:Int=0):Int={
   if(readFrom>s.length-1){
        level
    }else if(s(readFrom).toString!="."){
       level
   }else{
        countLevel(s,readFrom+1,level+1)
    }
}

def isOperator(s:String):Boolean={
    s.last.toString=="*" || s.last.toString =="+"
}
def extractOperator(s:String):Array[Int]=>Int={
    s.last.toString match {
        case "*" => (arr:Array[Int])=>arr.foldLeft(1){case (acc,v)=>acc*v}
        case "+" => (arr:Array[Int])=>arr.foldLeft(0){case (acc,v)=>acc+v}
    }
}
val identity = (arr:Array[Int])=>arr.head

def numerify(s:String):Int={
    s.dropWhile(_.toString==".").toInt
}

defined function countLevel
defined function isOperator
defined function extractOperator
identity: Array[Int] => Int = ammonite.$sess.cmd23$Helper$$Lambda$2386/0x0000000100c28840@1b96879c
defined function numerify

> 数学では，一般的に括弧を使って計算の順序を表現する． たとえば，7 × (3 + 2) は 3 + 2 を先に計算して， その結果を 7 に掛ける式であって，7 × 3 の結果に 2 を足す式ではない． しかし，世の中には括弧の存在を快く思わない人たちがいる． 国際反括弧会議 (International Counter of Parentheses Council) では，括弧を使わない記法を世界標準にしようとしている． 彼等は日夜括弧を使わない記法を研究している．

>，括弧を使わない新しい記法を考案した． 博士が考案した記法では，加算演算子 (+)，乗算演算子 (*)，あるいは整数からなる行を並べて式を表す． 式は，整数ひとつか，被演算子への演算子の適用のどちらかである． 整数は十進数で 1 行に表記される． 演算子の適用の場合は，演算子の行を先に書き，直後の行から演算子が適用されるふたつ以上の被演算子を並べて書くことで表現する．被演算子は再帰的に同じ記法で表記した式である． 被演算子が演算子の適用である場合，被演算子が複数の行にわたることになる．

>式はいくらでも入れ子になるので，どの演算子がどの被演算子に適用されるのかをわかるようにしなければならない． そのために，式には入れ子レベルを考える． 最上位の式は入れ子レベル 0 である． レベル n の式が演算子適用の場合，その被演算子はレベル n + 1 の式になる． 式の最初の行は入れ子レベルの数だけのピリオド (.) から始める．

>例えば，通常の数学で 2 + 3 と書く式は図1 のように書く． 演算子はふたつ以上の被演算子に適用でき，+ と * はそれぞれ全ての被演算子の総和や，全ての被演算子を掛け合わせる演算を表す． 例えば，図2 は 2 と 3 と 4 ?を掛け合わせる式である． 複雑な例では，通常の数学での (2 + 3 + 4) × 5 は図3 のように書け， (2 + 3) × 4 × 5 は図4 のように書ける．

>+
.2
.3
図1: 2 + 3
*
.2
.3
.4
図2: 2, 3, 4 を全て掛ける式
*
.+
..2
..3
..4
.5
図3: (2 + 3 + 4) × 5
*
.+
..2
..3
.4
.5
図4: (2 + 3) × 4 × 5
あなたの仕事は，博士を手伝ってこの記法で書かれた式の値を計算するプログラムを作ることだ．

> from icpc 2015

In [20]:
def calc(
    lines:Array[String],
    op:Array[Int]=>Int=identity,
    args:ArrayBuffer[Int]=ArrayBuffer[Int](),
    readFrom:Int=0,level:Int=0
):Int={
    if(readFrom>lines.length-1){
        op(args.toArray)
    }
    if(countLevel(lines(readFrom)) < level){
        if(isOperator(lines(readFrom))){
            op(args.toArray)+ calc(lines,extractOperator(lines(readFrom)),ArrayBuffer[Int](),readFrom+1,countLevel(lines(readFrom)))
        }else{
            op(args.toArray)+ numerify(lines(readFrom))
        }
    }else if (isOperator(lines(readFrom))){
        val r = calc(lines,extractOperator(lines(readFrom)),ArrayBuffer[Int](),readFrom+1,level+1)
        args.append(r)
        op(args.toArray)
    }else{
        args.append(numerify(lines(readFrom)))
        calc(lines,op,args,readFrom+1,level)
        
    }
}

defined function calc

In [21]:
/*sample inputs
1
9
4
+
.1
.2
.3
9
+
.0
.+
..*
...1
...*
....1
....2
..0
10
+
.+
..6
..2
.+
..1
..*
...7
...6
.3
0*/
val in="""+
.0
.+
..*
...1
...*
....1
....2
..0"""
calc(in.split("\n"))

in: String = """+
.0
.+
..*
...1
...*
....1
....2
..0"""
res20_1: Int = 2

stackでqueueを実装する

- 方針1: スタックに追加する際にQueueの順番に取り出せるように順序を整理する.
- 方針2: 一方のスタックAにFILOで要素を追加する. はじめてpopが呼ばれたら他方のスタックBにすべて移動する. このときBではFIFO順になっているので残りの要素はpopでO(1)で取り出せる. 新しい要素はAにFILOで追加される. Bが空になったらAからBに要素を再度移動させればいい.

In [ ]:
import scala.collection.mutable.ArrayStack
trait StaQueue[T]{
    //* push element to end
    def push(e:T):Unit
    //* pop element from first
    def pop():Option[T]
    // returns the first element of StaQueue
    def peek():Option[T]
    // returns whether the StaQueue is empty.
    def isEmpty():Boolean
}
// push O(n),pop O(1)
class NaiveStaQueue[T] extends StaQueue[T]{
    // store the first element so that we do not have to peek stack every time StaQueue.peek is called.
    private var front: Option[T] = None
    // Every element is saved in s1.
    private var s1:ArrayStack[T] = ArrayStack()
    // We make use of this auxiliary stack to arrange s1 elements in FIFO order.
    private var s2:ArrayStack[T] = ArrayStack()
    // newest element must be pushed to the bottom of the stack.
    
    override def push(e:T):Unit = {
        // first, we temporarily move all elements in s1 to s2.
        if(s1.isEmpty){
            front = Some(e)
            s1.push(e)
        }else{
            while(!s1.isEmpty){
                s2.push(s1.pop)
            }
            // then, put new element in the bottom of s1.
            s1.push(e)
            // then, move all elements back to s1.
            while(!s2.isEmpty){
                s1.push(s2.pop)
            }
        }
        
    }

    override def pop():Option[T] = {
        front match {
            case Some(e) =>
              s1.pop()
              front = if(s1.isEmpty) None else Some(s1.top) 
              Some(e)
            case _ => None
        }
    }
    
    override def peek():Option[T] = front
    
    override def isEmpty():Boolean = s1.isEmpty
}

class BetterStaQueue[T] extends StaQueue[T]{
    private var front: Option[T] = None
    private var s1:ArrayStack[T] = ArrayStack()
    private var s2:ArrayStack[T] = ArrayStack()
    
    override def push(e:T):Unit ={
        if(isEmpty){
            front = Some(e)
        }
        s1.push(e)
    }
    
    override def pop():Option[T]={
        // If s2 is empty, move all elements to s2 to arrange elements in FIFO order.
        if(s2.isEmpty){
            while(!s1.isEmpty){
                s2.push(s1.pop)
            }    
        }
        // now, we have FIFO ordered s2.
        if(s2.isEmpty){ // when both s1 and s2 are empty, front is None. 
            front = None
            front
        }else {
            val top = s2.pop
            front = if(s2.isEmpty) None else Some(s2.top)
            Some(top)             
        } 
    }
    
    
    override def peek():Option[T] = {
        if(!s2.isEmpty){
            Some(s2.top)
        }else{
            front
        }
        
    }
    
    override def isEmpty():Boolean = s1.isEmpty & s2.isEmpty

}


In [7]:
import scala.collection.mutable.ArrayBuffer

def pascalsTriangle(n:Int,from:Int=0,prevs:ArrayBuffer[Array[Int]]=ArrayBuffer(Array(1))):Array[Array[Int]]={
    if(from==n-1){
        return prevs.toArray
    }else{
        val prev = prevs(from)
        var a :ArrayBuffer[Int]= ArrayBuffer()
        a.append(1)
        if(prev.length>=2){
            for(i <- 0 to prev.length -2){
                a.append(prev(i)+prev(i+1))
            }
        }
 
        a.append(1)
        prevs.append(a.toArray)
        pascalsTriangle(n,from+1,prevs)
    
    }
}
pascalsTriangle(10).map(_.toList).toList

import scala.collection.mutable.ArrayBuffer


defined function pascalsTriangle
res6_2: List[List[Int]] = List(
  List(1),
  List(1, 1),
  List(1, 2, 1),
  List(1, 3, 3, 1),
  List(1, 4, 6, 4, 1),
  List(1, 5, 10, 10, 5, 1),
  List(1, 6, 15, 20, 15, 6, 1),
  List(1, 7, 21, 35, 35, 21, 7, 1),
  List(1, 8, 28, 56, 70, 56, 28, 8, 1),
  List(1, 9, 36, 84, 126, 126, 84, 36, 9, 1)
)

In [14]:
def mergeTwoSortedArray(arr:Array[Int],arr2:Array[Int],result:ArrayBuffer[Int]=ArrayBuffer()):Array[Int]={
    if(arr.isEmpty){
        result.appendAll(arr2.takeWhile(_!=0))
        result.toArray
    }else if(arr2.isEmpty){
        result.appendAll(arr.takeWhile(_!=0))
        result.toArray
    }else{
        if(arr(0)>arr2(0)){
            if(arr2(0)!=0){
                result.append(arr2(0))
            }
            mergeTwoSortedArray(arr,arr2.slice(1,arr2.length),result)
        }else{
            if(arr(0)!=0){
                    result.append(arr(0))

            }
            mergeTwoSortedArray(arr.slice(1,arr.length),arr2,result)
        }
    }
}
mergeTwoSortedArray(Array(1,3,5,9,0,0,0),Array(2,4,5,6,0))

defined function mergeTwoSortedArray
res13_1: Array[Int] = Array(1, 2, 3, 4, 5, 5, 6, 9)